In [ ]:
#@title Importing libraries
# Run if it gives you an error for any of these; FEEL FREE TO IMPORT MORE FOR EVERYONE TO USE!
!pip install numpy pandas scipy matplotlib seaborn plotly scikit-learn tensorflow torch nltk spacy geopandas folium pillow opencv-python pyspark dask

# Data manipulation and analysis
import numpy as np
import pandas as pd
import scipy as sp

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine learning
import sklearn
import tensorflow as tf
import torch

# Statistical analysis
import statsmodels.api as sm

# Natural Language Processing
import nltk
import spacy

# Geospatial analysis
import geopandas as gpd
import folium


In [ ]:
#@title importing datasets
!pip install -U -q PyDrive

import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_dataset(file_id, dataframe_name):
    # Authenticate and create the PyDrive client
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Download the file
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(f'{dataframe_name}.csv')

    # Read the CSV file into a DataFrame
    df = pd.read_csv(f'{dataframe_name}.csv')

    return df

To import a dataset, make sure it's a csv, upload to drive, copy the drive link, extract the id:

https://drive.google.com/open?id=1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK&usp=drive_copy means that the id is 1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK


plug into
name = import_dataset("id", "name")



In [ ]:
#@title PROVIDED DATASETS
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")
meat_stats_slaughter_weights = import_dataset("1IHd-9p3aDoxbER3oBCZqBY23VVD5LOnu", "meat_stats_slaughter_weights")
meat_stats_slaughter_counts = import_dataset("1Eo58CgEeIk7Hnw_7I2yY46Ng6qiBLyeX", "meat_stats_slaughter_counts")
meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
all_commodities = import_dataset("1E8ELVRv2OFMbXwWYp2XVE1wH6Or6kPkB", "all_commodities")
acs5yr = import_dataset("1JFpNj9SlUiWwZTFc-pnlggKu6T7h_urP", "acs5yr")


In [ ]:
#@title EXTERNAL DATASETS
agri_variable_list = import_dataset("1Y6Dv0yA8UXN5V5NhSdwUpT7JtzKj3oWm", "agri_variable_list")
agri_supplemental_county = import_dataset("1eM9e49xO841V-iR7wvFl1DPE-I8IRAKl", "agri_supplemental_county")
agri_supplemental_state = import_dataset("1BJsIiC3yPwtps1dA_-Ex-rUKzurce4c6", "agri_supplemental_state")
agri_state_and_county = import_dataset("1OFtRcynXKbMHFezP4ZLE5DEp6QUIGnHF", "agri_state_and_county")


employment = import_dataset("1x2uh0JILYFsarSP1h6Dtars79CtVYIP2", "employment")
unemployment = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

In [ ]:
health_region = import_dataset("18Y813FCdS2dXJSxOxUwcyVaAvwIxbOx-", "health_region")
social_determinants_of_health = import_dataset("17enYtkP62akioKX4_W9aup3pIiOyuwM8", "social_determinants_of_health")

In [ ]:
# utf-8 encoding
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

try:
    us_census_2020_2023 = import_dataset("1JHgbYOHYHR-Bdd1-NAv6laH8xMFZQJTq", "us_census_2020_2023", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2020_2023 with 'latin1' encoding")

try:
    us_census_2010_2020 = import_dataset("1aFonlFnV2dQKS4VgO4kgmsUaxYctWp8e", "us_census_2010_2020", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2010_2020 with 'latin1' encoding")

try:
    us_census_2000_2010 = import_dataset("1pWz7cBJmTBitrY6GJ_mwyISFijdbDIbK", "us_census_2000_2010", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2000_2010 with 'latin1' encoding")

In [ ]:
employment.head()

In [ ]:
#@title WRANGLING
#FOR all_commodites.csv !!!!

#fill missing 'Commodity' values since theyre fucking with us on corn
all_commodities['Commodity'] = all_commodities['Commodity'].fillna(method='ffill')

# Fill missing 'Unit' values based on 'Commodity'
all_commodities.loc[all_commodities['Commodity'] == 'Corn', 'Unit'] = 'Dollar Per Metric Tonne'
all_commodities.loc[all_commodities['Commodity'].isin(['Coffee', 'Sugar']), 'Unit'] = 'Cents Per Pound'

def convert_to_cents_per_pound(row):
    if row['Commodity'] == 'Corn' and row['Unit'] == 'Dollar Per Metric Tonne':
        #conversion: 1 tonne = 2204.62 pounds, dollar = 100 cents
        return (row['Value'] / 2204.62) * 100
    else:
        return row['Value']

all_commodities['Standardized_Value'] = all_commodities.apply(convert_to_cents_per_pound, axis=1)

# Update the Unit for Corn to 'Cents Per Pound'
all_commodities.loc[all_commodities['Commodity'] == 'Corn', 'Unit'] = 'Cents Per Pound'

# Save the corrected CSV file (optional)
all_commodities.to_csv('all_commodities_corrected.csv', index=False)


#FOR ALL meat_stats DATASETS
meat_stats_meat_production = pd.read_csv("meat_stats_meat_production.csv")
meat_stats_meat_production['Date-Time'] = pd.to_datetime(meat_stats_meat_production['Year'].astype(str) + '-' + meat_stats_meat_production['Month'].astype(str).str.zfill(2) + '-01')
meat_stats_meat_production.set_index('Date-Time', inplace=True)
meat_stats_meat_production.sort_index(inplace=True)
meat_stats_meat_production = meat_stats_meat_production.drop(['Year', 'Month'], axis=1)




In [ ]:
#@title Import Proxy Loan Dataset
loans0 = import_dataset("1LUw8MtYYFNapLUmt-ISqk_YzglegoYut", "1991-1999")
loans1 = import_dataset("1xpgOe0W8HMQOtl3f7qnwmx-Xi9r1zcKd", "2000-2009")
loans2 = import_dataset("1uhW_2RkHtHXQxq4TpYq5UlcTg-4hrOvY", "2010-2019")
loans3 = import_dataset("11RhgGiwaiXnROI4ZBxIHfetVK_03urzz", "2010-present")
loans0 = loans0[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans1 = loans1[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans2 = loans2[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans3 = loans3[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]

# Can use these regex's to sort using str.contains if you have questions lmk, case-insensitive
chain_names = ['POPEYE', 'starbucks', 'checkers', 'MCDONALD', 'del taco', ' qdoba', 'domino\'s', 'dutch bro', 'TACO BELL', 'DAIRY QUEEN', 'WAFFLE HOUSE', 'FRIED', 'KENTUCKY', 'BURGER', 'pizza', 'Arctic Circle', 'Chili', 'sonic drive', 'jack in the box', 'panda express', 'pei wei','papa john', 'little caesar', 'wendy\'s', 'wingstop', 'zaxby', 'jimmy john', 'five guys', 'hardee', 'bojangle', 'carl\'s jr', 'dunkin', 'krispy kreme', 'el pollo loco', 'shake shack', 'baskin robbins', 'church\'s chicken', 'papa murphy\'s', 'moe\s', 'freddy\'s frozen']
chain_regex = '|'.join(chain_names)
grocery_indicators = ['Grocery', 'Food store', 'bodega', 'food mart']
grocery_regex = '|'.join(grocery_indicators)

In [ ]:
#@title Pipeline for Getting Data (Example here based on Fiscal Year)
total_loans = pd.concat([loans0, loans1], ignore_index=False)
total_loans = pd.concat([total_loans, loans2], ignore_index=False)
total_loans = pd.concat([total_loans, loans3], ignore_index=False)

chains = total_loans[total_loans['BorrName'].str.contains(chain_regex, case=False, na=False)]
chain_year = chains['ApprovalFiscalYear'].value_counts(dropna=False)

# Chain_df is the full dataframe of loans given to top-50 chains
chain_df = chain_year.reset_index()
chain_df.sort_values('ApprovalFiscalYear', inplace=True)

In [ ]:
#@title Code in report

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import pandas as pd

loans1 = pd.read_csv('2000-2009.csv')
loans2 = pd.read_csv('2010-2019.csv')

print(loans1.head())
print(loans2.head())

In [ ]:
# @title Emily's code - loan stuff

# Combine loan datasets into one DataFrame
columns = ['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription',
           'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']
loans0 = loans0[columns]
loans1 = loans1[columns]
loans2 = loans2[columns]
loans3 = loans3[columns]
total_loans = pd.concat([loans0, loans1, loans2, loans3], ignore_index=False)

# Ensure ApprovalFiscalYear is an integer
total_loans['ApprovalFiscalYear'] = total_loans['ApprovalFiscalYear'].astype(int)

# Load the FIPS to county mapping dataset from the GitHub URL
fips_url = 'https://github.com/ChuckConnell/articles/raw/master/fips2county.tsv'
fips_mapping = pd.read_csv(fips_url, sep='\t', dtype={'FIPS': str})

# Standardize the case and strip whitespace for consistent merging
total_loans['ProjectCounty'] = total_loans['ProjectCounty'].str.upper().str.strip()
fips_mapping['CountyName'] = fips_mapping['CountyName'].str.upper().str.strip()

# Print samples to verify the cleaning
print(total_loans[['ProjectCounty']].head())
print(fips_mapping[['CountyName']].head())

# Merge loan data with the FIPS mapping data
total_loans = total_loans.merge(fips_mapping, left_on='ProjectCounty', right_on='CountyName', how='left')

# Print the first few rows to verify the merge
print(total_loans[['ProjectCounty', 'StateFIPS', 'CountyFIPS', 'StateName', 'CountyName']].head())

# Rename the columns to STATE and COUNTY for consistency
total_loans.rename(columns={'StateFIPS': 'STATE', 'CountyFIPS': 'COUNTY'}, inplace=True)

# Convert STATE and COUNTY in total_loans to integers
total_loans['STATE'] = pd.to_numeric(total_loans['STATE'], errors='coerce').astype(pd.Int64Dtype())
total_loans['COUNTY'] = pd.to_numeric(total_loans['COUNTY'], errors='coerce').astype(pd.Int64Dtype())

# Prepare the census data for each period
census_2020_2023 = us_census_2020_2023[['STATE', 'COUNTY', 'POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'POPESTIMATE2023']]
census_2010_2020 = us_census_2010_2020[['STATE', 'COUNTY', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019', 'POPESTIMATE2020']]
census_2000_2010 = us_census_2000_2010[['STATE', 'COUNTY', 'POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'POPESTIMATE2010']]

# Melt the census data to have one population estimate per row
census_2020_2023 = census_2020_2023.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2020_2023['Year'] = census_2020_2023['Year'].str.extract('(\d+)').astype(int)

census_2010_2020 = census_2010_2020.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2010_2020['Year'] = census_2010_2020['Year'].str.extract('(\d+)').astype(int)

census_2000_2010 = census_2000_2010.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2000_2010['Year'] = census_2000_2010['Year'].str.extract('(\d+)').astype(int)

# Combine all census data
census_data = pd.concat([census_2020_2023, census_2010_2020, census_2000_2010])

# Verify the state and county columns are in the correct format
print(total_loans[['STATE', 'COUNTY']].drop_duplicates().head())
print(census_data[['STATE', 'COUNTY']].drop_duplicates().head())

# Merge loan data with census data
merged_loans = total_loans.merge(census_data, left_on=['STATE', 'COUNTY', 'ApprovalFiscalYear'], right_on=['STATE', 'COUNTY', 'Year'], how='left', indicator=True)

# Check the merge result
print(merged_loans['_merge'].value_counts())

# Filter out only the successfully merged rows for further analysis
total_loans = merged_loans[merged_loans['_merge'] == 'both']
total_loans.drop(columns=['_merge'], inplace=True)

# Recalculate LoansPerCapita
total_loans['LoansPerCapita'] = total_loans.groupby(['STATE', 'COUNTY', 'ApprovalFiscalYear'])['BorrName'].transform('count') / total_loans['Population']

# Filter for fast food chains and grocery stores
chain_names = ['POPEYE', 'starbucks', 'checkers', 'MCDONALD', 'del taco', 'qdoba', 'domino\'s', 'dutch bro', 'TACO BELL', 'DAIRY QUEEN', 'WAFFLE HOUSE', 'FRIED', 'KENTUCKY', 'BURGER', 'pizza', 'Arctic Circle', 'Chili', 'sonic drive', 'jack in the box', 'panda express', 'pei wei','papa john', 'little caesar', 'wendy\'s', 'wingstop', 'zaxby', 'jimmy john', 'five guys', 'hardee', 'bojangle', 'carl\'s jr', 'dunkin', 'krispy kreme', 'el pollo loco', 'shake shack', 'baskin robbins', 'church\'s chicken', 'papa murphy\'s', 'moe\s', 'freddy\'s frozen']
chain_regex = '|'.join(chain_names)
grocery_indicators = ['Grocery', 'Food store', 'bodega', 'food mart']
grocery_regex = '|'.join(grocery_indicators)

fast_food_loans = total_loans[total_loans['FranchiseName'].str.contains(chain_regex, case=False, na=False)]
grocery_loans = total_loans[total_loans['NaicsDescription'].str.contains(grocery_regex, case=False, na=False)]

# Recalculate trends
fast_food_trends = fast_food_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()
grocery_trends = grocery_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()

# Plot the trends
plt.figure(figsize=(12, 6))
plt.plot(fast_food_trends, label='Fast Food Chains')
plt.plot(grocery_trends, label='Grocery Stores')
plt.xlabel('Year')
plt.ylabel('Loans Per Capita')
plt.title('Trends in Loan Approvals Per Capita for Fast Food Chains vs Grocery Stores')
plt.legend()
plt.show()

# Calculate top 10 counties by loans per capita
top_10_counties = total_loans.groupby('COUNTY')['LoansPerCapita'].mean().sort_values(ascending=False).head(10)
top_10_counties.plot(kind='bar')
plt.xlabel('County')
plt.ylabel('Loans Per Capita')
plt.title('Top 10 Counties by Loans Per Capita')
plt.show()

In [ ]:
# Ensure ApprovalFiscalYear is an integer
total_loans['ApprovalFiscalYear'] = pd.to_numeric(total_loans['ApprovalFiscalYear'], errors='coerce').astype(int)

# Standardize the case and strip whitespace for consistent merging
total_loans['ProjectCounty'] = total_loans['ProjectCounty'].str.upper().str.strip()
fips_mapping['CountyName'] = fips_mapping['CountyName'].str.upper().str.strip()

# Merge loan data with the FIPS mapping data
total_loans = total_loans.merge(fips_mapping, left_on='ProjectCounty', right_on='CountyName', how='left')

# Rename the columns to STATE and COUNTY for consistency
total_loans.rename(columns={'StateFIPS': 'STATE', 'CountyFIPS': 'COUNTY'}, inplace=True)

# Convert STATE and COUNTY in total_loans to integers
total_loans['STATE'] = pd.to_numeric(total_loans['STATE'], errors='coerce').astype(pd.Int64Dtype())
total_loans['COUNTY'] = pd.to_numeric(total_loans['COUNTY'], errors='coerce').astype(pd.Int64Dtype())

# Filter Mississippi loan data
mississippi_loans = total_loans[total_loans['StateName'] == 'MISSISSIPPI']
print("Mississippi loans data before merge:")
print(mississippi_loans[['STATE', 'COUNTY', 'ApprovalFiscalYear']].drop_duplicates().head())

# Verify unique values in Mississippi loan data
print("Unique STATE and COUNTY values in mississippi_loans:")
print(mississippi_loans[['STATE', 'COUNTY']].drop_duplicates())

# Filter Mississippi data from census_data
mississippi_census_data = census_data[census_data['STATE'] == 28]  # Mississippi's FIPS code is 28
print("Unique COUNTY values in Mississippi census_data:")
print(mississippi_census_data[['COUNTY']].drop_duplicates().head())

# Merge loan data with census data
mississippi_loans = mississippi_loans.merge(mississippi_census_data, left_on=['STATE', 'COUNTY', 'ApprovalFiscalYear'], right_on=['STATE', 'COUNTY', 'Year'], how='left', indicator=True)

# Check the merge result
print("Merge result for Mississippi loans after specific filtering:")
print(mississippi_loans['_merge'].value_counts())

# Filter out only the successfully merged rows for further analysis
mississippi_loans = mississippi_loans[mississippi_loans['_merge'] == 'both']
mississippi_loans.drop(columns=['_merge'], inplace=True)

# Recalculate LoansPerCapita
mississippi_loans['LoansPerCapita'] = mississippi_loans.groupby(['STATE', 'COUNTY', 'ApprovalFiscalYear'])['BorrName'].transform('count') / mississippi_loans['Population']

# Filter for fast food chains and grocery stores in Mississippi
fast_food_loans = mississippi_loans[mississippi_loans['FranchiseName'].str.contains(chain_regex, case=False, na=False)]
grocery_loans = mississippi_loans[mississippi_loans['NaicsDescription'].str.contains(grocery_regex, case=False, na=False)]

# Recalculate trends
fast_food_trends = fast_food_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()
grocery_trends = grocery_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()

# Plot the trends for Mississippi
plt.figure(figsize=(12, 6))
plt.plot(fast_food_trends, label='Fast Food Chains')
plt.plot(grocery_trends, label='Grocery Stores')
plt.xlabel('Year')
plt.ylabel('Loans Per Capita')
plt.title('Trends in Loan Approvals Per Capita for Fast Food Chains vs Grocery Stores in Mississippi')
plt.legend()
plt.show()

# Calculate top 10 counties by loans per capita in Mississippi
top_10_counties = mississippi_loans.groupby('COUNTY')['LoansPerCapita'].mean().sort_values(ascending=False).head(10)
top_10_counties.plot(kind='bar')
plt.xlabel('County')
plt.ylabel('Loans Per Capita')
plt.title('Top 10 Counties by Loans Per Capita in Mississippi')
plt.show()


In [ ]:
# Mississippi stuff

# Combine loan datasets into one DataFrame
columns = ['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription',
           'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']
loans0 = loans0[columns]
loans1 = loans1[columns]
loans2 = loans2[columns]
loans3 = loans3[columns]
total_loans = pd.concat([loans0, loans1, loans2, loans3], ignore_index=False)

# Ensure ApprovalFiscalYear is an integer
total_loans['ApprovalFiscalYear'] = total_loans['ApprovalFiscalYear'].astype(int)

# Load the FIPS to county mapping dataset from the GitHub URL
fips_url = 'https://github.com/ChuckConnell/articles/raw/master/fips2county.tsv'
fips_mapping = pd.read_csv(fips_url, sep='\t', dtype={'FIPS': str})

# Standardize the case and strip whitespace for consistent merging
total_loans['ProjectCounty'] = total_loans['ProjectCounty'].str.upper().str.strip()
fips_mapping['CountyName'] = fips_mapping['CountyName'].str.upper().str.strip()

# Merge loan data with the FIPS mapping data
total_loans = total_loans.merge(fips_mapping, left_on='ProjectCounty', right_on='CountyName', how='left')

# Rename the columns to STATE and COUNTY for consistency
total_loans.rename(columns={'StateFIPS': 'STATE', 'CountyFIPS': 'COUNTY'}, inplace=True)

# Convert STATE and COUNTY in total_loans to integers
total_loans['STATE'] = pd.to_numeric(total_loans['STATE'], errors='coerce').astype(pd.Int64Dtype())
total_loans['COUNTY'] = pd.to_numeric(total_loans['COUNTY'], errors='coerce').astype(pd.Int64Dtype())

# Prepare the census data for each period
census_2020_2023 = us_census_2020_2023[['STATE', 'COUNTY', 'POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022', 'POPESTIMATE2023']]
census_2010_2020 = us_census_2010_2020[['STATE', 'COUNTY', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019', 'POPESTIMATE2020']]
census_2000_2010 = us_census_2000_2010[['STATE', 'COUNTY', 'POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'POPESTIMATE2010']]

# Melt the census data to have one population estimate per row
census_2020_2023 = census_2020_2023.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2020_2023['Year'] = census_2020_2023['Year'].str.extract('(\d+)').astype(int)

census_2010_2020 = census_2010_2020.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2010_2020['Year'] = census_2010_2020['Year'].str.extract('(\d+)').astype(int)

census_2000_2010 = census_2000_2010.melt(id_vars=['STATE', 'COUNTY'], var_name='Year', value_name='Population')
census_2000_2010['Year'] = census_2000_2010['Year'].str.extract('(\d+)').astype(int)

# Combine all census data
census_data = pd.concat([census_2020_2023, census_2010_2020, census_2000_2010])

# Filter the data for Mississippi (State FIPS code 28)
mississippi_loans = total_loans[total_loans['STATE'] == 28]
mississippi_census_data = census_data[census_data['STATE'] == 28]

# Merge loan data with census data for Mississippi
merged_loans = mississippi_loans.merge(mississippi_census_data, left_on=['STATE', 'COUNTY', 'ApprovalFiscalYear'], right_on=['STATE', 'COUNTY', 'Year'], how='left', indicator=True)

# Check the merge result
print(merged_loans['_merge'].value_counts())

# Filter out only the successfully merged rows for further analysis
total_loans = merged_loans[merged_loans['_merge'] == 'both']
total_loans.drop(columns=['_merge'], inplace=True)

# Recalculate LoansPerCapita
total_loans['LoansPerCapita'] = total_loans.groupby(['STATE', 'COUNTY', 'ApprovalFiscalYear'])['BorrName'].transform('count') / total_loans['Population']

# Filter for fast food chains and grocery stores
chain_names = ['POPEYE', 'starbucks', 'checkers', 'MCDONALD', 'del taco', 'qdoba', 'domino\'s', 'dutch bro', 'TACO BELL', 'DAIRY QUEEN', 'WAFFLE HOUSE', 'FRIED', 'KENTUCKY', 'BURGER', 'pizza', 'Arctic Circle', 'Chili', 'sonic drive', 'jack in the box', 'panda express', 'pei wei','papa john', 'little caesar', 'wendy\'s', 'wingstop', 'zaxby', 'jimmy john', 'five guys', 'hardee', 'bojangle', 'carl\'s jr', 'dunkin', 'krispy kreme', 'el pollo loco', 'shake shack', 'baskin robbins', 'church\'s chicken', 'papa murphy\'s', 'moe\s', 'freddy\'s frozen']
chain_regex = '|'.join(chain_names)
grocery_indicators = ['Grocery', 'Food store', 'bodega', 'food mart']
grocery_regex = '|'.join(grocery_indicators)

fast_food_loans = total_loans[total_loans['FranchiseName'].str.contains(chain_regex, case=False, na=False)]
grocery_loans = total_loans[total_loans['NaicsDescription'].str.contains(grocery_regex, case=False, na=False)]

# Recalculate trends
fast_food_trends = fast_food_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()
grocery_trends = grocery_loans.groupby('ApprovalFiscalYear')['LoansPerCapita'].mean()

# Plot the trends
plt.figure(figsize=(12, 6))
plt.plot(fast_food_trends, label='Fast Food Chains')
plt.plot(grocery_trends, label='Grocery Stores')
plt.xlabel('Year')
plt.ylabel('Loans Per Capita')
plt.title('Trends in Loan Approvals Per Capita for Fast Food Chains vs Grocery Stores in Mississippi')
plt.legend()
plt.show()

# Calculate top 10 counties by loans per capita
top_10_counties = total_loans.groupby('COUNTY')['LoansPerCapita'].mean().sort_values(ascending=False).head(10)
top_10_counties.plot(kind='bar')
plt.xlabel('County')
plt.ylabel('Loans Per Capita')
plt.title('Top 10 Counties by Loans Per Capita in Mississippi')
plt.show()


In [ ]:
# @title Emily's code - county stuff

print("agri_variable_list:")
print(agri_variable_list.info())
print(agri_variable_list.head(), "\n")

print("agri_supplemental_county:")
print(agri_supplemental_county.info())
print(agri_supplemental_county.head(), "\n")

print("agri_supplemental_state:")
print(agri_supplemental_state.info())
print(agri_supplemental_state.head(), "\n")

print("agri_state_and_county:")
print(agri_state_and_county.info())
print(agri_state_and_county.head(), "\n")

print("Unique Variable Codes in agri_variable_list:")
print(agri_variable_list['Variable_Code'].unique(), "\n")

print("Unique Variable Codes in agri_supplemental_county:")
print(agri_supplemental_county['Variable_Code'].unique(), "\n")

print("Unique Variable Codes in agri_supplemental_state:")
print(agri_supplemental_state['Variable_Code'].unique(), "\n")

print("Unique Variable Codes in agri_state_and_county:")
print(agri_state_and_county['Variable_Code'].unique(), "\n")

unique_variable_codes = agri_state_and_county['Variable_Code'].unique()

# for code in unique_variable_codes:
#     print(code)

# looking into these 3 things:
# 1. Obesity rates
# 2. Food access indicators
# 3. Socioeconomic factors

import pandas as pd
import matplotlib.pyplot as plt

# Filter relevant data
relevant_variable_codes = [
    'PCT_OBESE_ADULTS12', 'PCT_OBESE_ADULTS17',  # Obesity rates
    'LACCESS_POP10', 'LACCESS_POP15',            # Food access
    'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15',
    'MEDHHINC15', 'POVRATE15', 'CHILDPOVRATE15'  # Socioeconomic factors
]

relevant_data = agri_state_and_county[agri_state_and_county['Variable_Code'].isin(relevant_variable_codes)]
pivot_data = relevant_data.pivot_table(index=['FIPS', 'State', 'County'], columns='Variable_Code', values='Value')
pivot_data.reset_index(inplace=True)

# plot
def plot_obesity_rates(data):
    plt.figure(figsize=(10, 5))
    for column in ['PCT_OBESE_ADULTS12', 'PCT_OBESE_ADULTS17']:
        if column in data.columns:
            plt.plot(data['FIPS'], data[column], linestyle='-', label=column)
    plt.xlabel('County FIPS Code')
    plt.ylabel('Percentage')
    plt.title('Obesity Rates Over Time')
    plt.legend()
    plt.show()

def plot_food_access(data):
    plt.figure(figsize=(10, 5))
    for column in ['LACCESS_POP10', 'LACCESS_POP15', 'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15']:
        if column in data.columns:
            plt.plot(data['FIPS'], data[column], linestyle='-', label=column)
    plt.xlabel('County FIPS Code')
    plt.ylabel('Value')
    plt.title('Food Access Indicators Over Time')
    plt.legend()
    plt.show()

def plot_socioeconomic_factors(data):
    plt.figure(figsize=(10, 5))
    for column in ['MEDHHINC15', 'POVRATE15', 'CHILDPOVRATE15']:
        if column in data.columns:
            plt.plot(data['FIPS'], data[column], linestyle='-', label=column)
    plt.xlabel('County FIPS Code')
    plt.ylabel('Value')
    plt.title('Socioeconomic Factors')
    plt.legend()
    plt.show()

plot_obesity_rates(pivot_data)
plot_food_access(pivot_data)
plot_socioeconomic_factors(pivot_data)

# summary stats
summary_stats = pivot_data.describe()
print(summary_stats)

In [ ]:
#@title Emily's code - merge loan data and agri data - I need to fix this

total_loans['ProjectCounty'] = total_loans['ProjectCounty'].str.lower()
pivot_data['ProjectCounty'] = pivot_data['ProjectCounty'].str.lower()

if 'State' in total_loans.columns and 'State' in pivot_data.columns:
    merge_columns = ['ProjectCounty', 'State']
else:
    merge_columns = ['ProjectCounty']

merged_data = pd.merge(total_loans, pivot_data, on=merge_columns, how='inner')

print(merged_data.head())
print(merged_data.describe())
print(merged_data.isnull().sum())

# plots
# def plot_obesity_vs_loans(data):
#     plt.figure(figsize=(10, 5))
#     if 'PCT_OBESE_ADULTS12' in data.columns:
#         plt.scatter(data['PCT_OBESE_ADULTS12'], data['ApprovalFiscalYear'], alpha=0.5, label='Obesity Rate 2012')
#     if 'PCT_OBESE_ADULTS17' in data.columns:
#         plt.scatter(data['PCT_OBESE_ADULTS17'], data['ApprovalFiscalYear'], alpha=0.5, label='Obesity Rate 2017')
#     plt.xlabel('Obesity Rate (%)')
#     plt.ylabel('Approval Fiscal Year')
#     plt.title('Obesity Rates vs. Loan Approval Years')
#     plt.legend()
#     plt.show()

# def plot_food_access_vs_loans(data):
#     plt.figure(figsize=(10, 5))
#     if 'LACCESS_POP10' in data.columns:
#         plt.scatter(data['LACCESS_POP10'], data['ApprovalFiscalYear'], alpha=0.5, label='Low Access Pop 2010')
#     if 'LACCESS_POP15' in data.columns:
#         plt.scatter(data['LACCESS_POP15'], data['ApprovalFiscalYear'], alpha=0.5, label='Low Access Pop 2015')
#     plt.xlabel('Low Access Population')
#     plt.ylabel('Approval Fiscal Year')
#     plt.title('Food Access vs. Loan Approval Years')
#     plt.legend()
#     plt.show()

# def plot_socioeconomic_vs_loans(data):
#     plt.figure(figsize=(10, 5))
#     if 'MEDHHINC15' in data.columns:
#         plt.scatter(data['MEDHHINC15'], data['ApprovalFiscalYear'], alpha=0.5, label='Median Household Income 2015')
#     if 'POVRATE15' in data.columns:
#         plt.scatter(data['POVRATE15'], data['ApprovalFiscalYear'], alpha=0.5, label='Poverty Rate 2015')
#     if 'CHILDPOVRATE15' in data.columns:
#         plt.scatter(data['CHILDPOVRATE15'], data['ApprovalFiscalYear'], alpha=0.5, label='Child Poverty Rate 2015')
#     plt.xlabel('Socioeconomic Factors')
#     plt.ylabel('Approval Fiscal Year')
#     plt.title('Socioeconomic Factors vs. Loan Approval Years')
#     plt.legend()
#     plt.show()

# if not merged_data.empty:
#     plot_obesity_vs_loans(merged_data)
#     plot_food_access_vs_loans(merged_data)
#     plot_socioeconomic_vs_loans(merged_data)
# else:
#     print("The merged data is empty. Check the county and state names for mismatches.")



In [ ]:
#@title aggregate data by Fiscal Year and calculate mean values
aggregated_data = merged_data.groupby('ApprovalFiscalYear').agg({
    'PCT_OBESE_ADULTS12': 'mean',
    'PCT_OBESE_ADULTS17': 'mean',
    'LACCESS_POP10': 'mean',
    'LACCESS_POP15': 'mean',
    'PCT_LACCESS_POP10': 'mean',
    'PCT_LACCESS_POP15': 'mean',
    'MEDHHINC15': 'mean',
    'POVRATE15': 'mean',
    'CHILDPOVRATE15': 'mean'
}).reset_index()

# plots
def plot_aggregated_obesity_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data['ApprovalFiscalYear'], data['PCT_OBESE_ADULTS12'], marker='o', linestyle='-', label='Obesity Rate 2012')
    plt.plot(data['ApprovalFiscalYear'], data['PCT_OBESE_ADULTS17'], marker='o', linestyle='-', label='Obesity Rate 2017')
    plt.xlabel('Approval Fiscal Year')
    plt.ylabel('Average Obesity Rate (%)')
    plt.title('Average Obesity Rates vs. Loan Approval Years')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_aggregated_food_access_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data['ApprovalFiscalYear'], data['LACCESS_POP10'], marker='o', linestyle='-', label='Low Access Pop 2010')
    plt.plot(data['ApprovalFiscalYear'], data['LACCESS_POP15'], marker='o', linestyle='-', label='Low Access Pop 2015')
    plt.plot(data['ApprovalFiscalYear'], data['PCT_LACCESS_POP10'], marker='o', linestyle='-', label='Low Access Pop % 2010')
    plt.plot(data['ApprovalFiscalYear'], data['PCT_LACCESS_POP15'], marker='o', linestyle='-', label='Low Access Pop % 2015')
    plt.xlabel('Approval Fiscal Year')
    plt.ylabel('Average Low Access Population')
    plt.title('Average Food Access vs. Loan Approval Years')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_aggregated_socioeconomic_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.plot(data['ApprovalFiscalYear'], data['MEDHHINC15'], marker='o', linestyle='-', label='Median Household Income 2015')
    plt.plot(data['ApprovalFiscalYear'], data['POVRATE15'], marker='o', linestyle='-', label='Poverty Rate 2015')
    plt.plot(data['ApprovalFiscalYear'], data['CHILDPOVRATE15'], marker='o', linestyle='-', label='Child Poverty Rate 2015')
    plt.xlabel('Approval Fiscal Year')
    plt.ylabel('Average Socioeconomic Factors')
    plt.title('Average Socioeconomic Factors vs. Loan Approval Years')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_aggregated_obesity_vs_loans(aggregated_data)
plot_aggregated_food_access_vs_loans(aggregated_data)
plot_aggregated_socioeconomic_vs_loans(aggregated_data)


In [ ]:
#@title need to fix
total_loans['ProjectCounty'] = total_loans['ProjectCounty'].str.lower()

pivot_data.reset_index(inplace=True)
if 'County' in pivot_data.columns:
    pivot_data['County'] = pivot_data['County'].str.lower()
    pivot_data.rename(columns={'County': 'ProjectCounty'}, inplace=True)
else:
    print("Error: 'County' column is missing in pivot_data")

print("Total Loans Columns:", total_loans.columns)
print("Pivot Data Columns:", pivot_data.columns)

if 'State' not in total_loans.columns:
    total_loans['State'] = 'unknown'

if 'State' not in pivot_data.columns:
    pivot_data['State'] = 'unknown'

merged_data = pd.merge(total_loans, pivot_data, on=['ProjectCounty'], how='inner')

print("Merged Data Head:")
print(merged_data.head())
print("Merged Data Describe:")
print(merged_data.describe())
print("Merged Data Null Values:")
print(merged_data.isnull().sum())

# aggregate
aggregated_data = merged_data.groupby('ProjectCounty').agg({
    'PCT_OBESE_ADULTS12': 'mean',
    'PCT_OBESE_ADULTS17': 'mean',
    'LACCESS_POP10': 'mean',
    'LACCESS_POP15': 'mean',
    'PCT_LACCESS_POP10': 'mean',
    'PCT_LACCESS_POP15': 'mean',
    'MEDHHINC15': 'mean',
    'POVRATE15': 'mean',
    'CHILDPOVRATE15': 'mean'
}).reset_index()

# plot
def plot_aggregated_obesity_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.scatter(data['PCT_OBESE_ADULTS12'], data['PCT_OBESE_ADULTS17'], alpha=0.5, label='Obesity Rate 2012 vs 2017')
    plt.xlabel('Obesity Rate 2012 (%)')
    plt.ylabel('Obesity Rate 2017 (%)')
    plt.title('Obesity Rates 2012 vs 2017 by County')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_aggregated_food_access_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.scatter(data['LACCESS_POP10'], data['LACCESS_POP15'], alpha=0.5, label='Low Access Pop 2010 vs 2015')
    plt.scatter(data['PCT_LACCESS_POP10'], data['PCT_LACCESS_POP15'], alpha=0.5, label='Low Access Pop % 2010 vs 2015')
    plt.xlabel('Low Access Population 2010')
    plt.ylabel('Low Access Population 2015')
    plt.title('Food Access 2010 vs 2015 by County')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_aggregated_socioeconomic_vs_loans(data):
    plt.figure(figsize=(10, 5))
    plt.scatter(data['MEDHHINC15'], data['POVRATE15'], alpha=0.5, label='Median Household Income vs Poverty Rate 2015')
    plt.scatter(data['MEDHHINC15'], data['CHILDPOVRATE15'], alpha=0.5, label='Median Household Income vs Child Poverty Rate 2015')
    plt.xlabel('Median Household Income 2015')
    plt.ylabel('Poverty Rate / Child Poverty Rate 2015')
    plt.title('Socioeconomic Factors 2015 by County')
    plt.legend()
    plt.grid(True)
    plt.show()

# plot
plot_aggregated_obesity_vs_loans(aggregated_data)
plot_aggregated_food_access_vs_loans(aggregated_data)
plot_aggregated_socioeconomic_vs_loans(aggregated_data)


In [ ]:
#@title Hasan
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from sklearn.preprocessing import StandardScaler


all_stocks = pd.read_csv(all_stocks.csv)
stock_descriptions = pd.read_csv(stock_descriptions.csv)
nutrition = pd.read_csv(nutrition.csv)
#useful functions
def check_stationarity(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    return result[1] <= 0.05  # Returns True if p-value <= 0.05 (stationary)

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}
    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]
    return cross_corr_lags

#data prep
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

fast_food_stock_prices['Date-Time'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices['Year'] = fast_food_stock_prices['Date-Time'].dt.year
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby('Year')['Close'].mean().reset_index()

obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]
obesity_by_year = obesity_data.groupby('YearStart')['Data_Value'].mean().reset_index()
obesity_by_year.columns = ['Year', 'ObesityRate']

#merging
merged_data = pd.merge(obesity_by_year, avg_fast_food_stock_prices_close, on='Year', how='inner')

print("\nStationarity Check:")
for column in ['ObesityRate', 'Close']:
    is_stationary = check_stationarity(merged_data[column])
    print(f"{column}: {'Stationary' if is_stationary else 'Non-stationary'}")

#standardizing
scaler = StandardScaler()
merged_data_standardized = pd.DataFrame(scaler.fit_transform(merged_data[['ObesityRate', 'Close']]),
                                        columns=['ObesityRate_std', 'Close_std'],
                                        index=merged_data.index)
merged_data_standardized['Year'] = merged_data['Year']

#plotting
fig, ax1 = plt.subplots()
ax1.set_xlabel('Year')
ax1.set_ylabel('Obesity Rate (%)', color='tab:blue')
ax1.plot(merged_data['Year'], merged_data['ObesityRate'], color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Average Stock Price for Fast Food Companies (USD)', color='tab:red')
ax2.plot(merged_data['Year'], merged_data['Close'], color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Obesity Rates and Fast Food Stock Prices Over Time')
plt.show()

#cross correlation
numeric_data = merged_data[['ObesityRate', 'Close']]
max_lag = 2  # Set max lag to 2 cause others arent relevant by inspection
cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='crest', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag} months)')
    plt.tight_layout()
    plt.show()

#growth rate
merged_data['ObesityRateGrowth'] = merged_data['ObesityRate'].pct_change()
merged_data['StockPriceGrowth'] = merged_data['Close'].pct_change()

avg_growth_rates = merged_data[['ObesityRateGrowth', 'StockPriceGrowth']].mean()
print("\nAverage Annual Growth Rates:")
print(avg_growth_rates)

#causality
print("\nGranger Causality Tests:")
max_lag = 4  #it bugs out for higher than this idk why but it shouldnt matter anyway since corerlation beyond 3 is low
variables = ['ObesityRate', 'Close']
for i in range(len(variables)):
    for j in range(len(variables)):
        if i != j:
            print(f"{variables[i]} -> {variables[j]}:")
            test_result = grangercausalitytests(merged_data[[variables[i], variables[j]]], maxlag=max_lag, verbose=False)
            for lag in range(1, max_lag + 1):  # Start from lag 1 to avoid lag 0
                p_value = test_result[lag][0]['ssr_ftest'][1]
                print(f"  Lag {lag}: p-value = {p_value:.4f}")


In [ ]:
#@title Hasan code 2: volume and volatility for fast food stocks; feel free to superpose whatever else you think should be compared in top of this
#additional analysis on stock volume trends
volume_trends = fast_food_stock_prices.groupby('Year')['Volume'].mean().reset_index()
plt.figure(figsize=(14, 7))
sns.lineplot(data=volume_trends, x='Year', y='Volume', color='purple')
plt.title('Average Trading Volume of Fast Food Stocks Over Time')
plt.xlabel('Year')
plt.ylabel('Average Volume')
plt.show()

#volatility analysis
fast_food_stock_prices['Daily_Return'] = fast_food_stock_prices.groupby('Ticker_Symbol')['Close'].pct_change()
volatility = fast_food_stock_prices.groupby(['Year', 'Ticker_Symbol'])['Daily_Return'].std().reset_index()
avg_volatility = volatility.groupby('Year')['Daily_Return'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(data=avg_volatility, x='Year', y='Daily_Return', color='orange')
plt.title('Average Volatility of Fast Food Stocks Over Time')
plt.xlabel('Year')
plt.ylabel('Average Daily Return Volatility')
plt.show()

In [ ]:
#@title Hasan 2.5 volatility and volume for entire market vs just fast food
#data prep
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

fast_food_stock_prices['Date-Time'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices['Year'] = fast_food_stock_prices['Date-Time'].dt.year

# volume
fast_food_volume_trends = fast_food_stock_prices.groupby('Year')['Volume'].mean().reset_index()
fast_food_volume_trends.columns = ['Year', 'FastFoodVolume']

# entire market
all_stocks['Date-Time'] = pd.to_datetime(all_stocks['Date-Time'])
all_stocks['Year'] = all_stocks['Date-Time'].dt.year
market_volume_trends = all_stocks.groupby('Year')['Volume'].mean().reset_index()
market_volume_trends.columns = ['Year', 'MarketVolume']

#fast food stocks
plt.figure(figsize=(14, 7))
plt.plot(fast_food_volume_trends['Year'], fast_food_volume_trends['FastFoodVolume'], color='purple', label='Fast Food Stocks')
plt.plot(market_volume_trends['Year'], market_volume_trends['MarketVolume'], color='blue', label='Entire Stock Market')
plt.title('Average Trading Volume Over Time')
plt.xlabel('Year')
plt.ylabel('Average Volume')
plt.legend()
plt.show()

#volatility
#fast food stocks
fast_food_stock_prices['Daily_Return'] = fast_food_stock_prices.groupby('Ticker_Symbol')['Close'].pct_change()
fast_food_volatility = fast_food_stock_prices.groupby(['Year', 'Ticker_Symbol'])['Daily_Return'].std().reset_index()
avg_fast_food_volatility = fast_food_volatility.groupby('Year')['Daily_Return'].mean().reset_index()
avg_fast_food_volatility.columns = ['Year', 'FastFoodVolatility']

#entire market
all_stocks['Daily_Return'] = all_stocks.groupby('Ticker_Symbol')['Close'].pct_change()
market_volatility = all_stocks.groupby(['Year', 'Ticker_Symbol'])['Daily_Return'].std().reset_index()
avg_market_volatility = market_volatility.groupby('Year')['Daily_Return'].mean().reset_index()
avg_market_volatility.columns = ['Year', 'MarketVolatility']

#plot
plt.figure(figsize=(14, 7))
plt.plot(avg_fast_food_volatility['Year'], avg_fast_food_volatility['FastFoodVolatility'], color='orange', label='Fast Food Stocks')
plt.plot(avg_market_volatility['Year'], avg_market_volatility['MarketVolatility'], color='green', label='Entire Stock Market')
plt.title('Average Volatility Over Time')
plt.xlabel('Year')
plt.ylabel('Average Daily Return Volatility')
plt.legend()
plt.show()

In [ ]:
#@title Hasan 3: commodities, meat production, and stocks
from sklearn.preprocessing import StandardScaler

#preparing data
fast_food_stock_prices
if 'Date-Time' in fast_food_stock_prices.columns:
    fast_food_stock_prices.set_index('Date-Time', inplace=True)
fast_food_monthly = fast_food_stock_prices['Close'].resample('M').mean().to_frame('Fast_Food_Stock_Price')

commodities = ['Corn', 'Coffee', 'Sugar']
commodity_prices = all_commodities[all_commodities['Commodity'].isin(commodities)]
commodity_prices['Date-Time'] = pd.to_datetime(commodity_prices['Date-Time'])
commodity_prices.set_index('Date-Time', inplace=True)

meat_production = meat_stats_meat_production.copy()

# meat_production['DateTime'] = pd.to_datetime(meat_production['Year'].astype(str) + '-' + meat_production['Month'].astype(str).str.zfill(2) + '-01')
# meat_production.set_index('Date-Time', inplace=True)
# meat_production.sort_index(inplace=True)
# meat_production = meat_production.drop(['Year', 'Month'], axis=1)


#pivot meat_production data to make merging easier later
meat_production_pivoted = meat_production.pivot_table(
    values='Production',
    index='Date-Time',
    columns=['Animal', 'Commercial or Federally Inspected', 'Type of Meat'],
    aggfunc='sum'
)

#flatten column names to make merging easier later
meat_production_pivoted.columns = [f"{animal}_{inspection}_{meat_type}" for animal, inspection, meat_type in meat_production_pivoted.columns]

#resample all data to monthly frequency
fast_food_monthly = fast_food_stock_prices['Close'].resample('M').mean().to_frame('Fast_Food_Stock_Price')
commodity_monthly = commodity_prices.pivot(columns='Commodity', values='Value').resample('M').mean()
meat_production_monthly = meat_production_pivoted.resample('M').sum()

#getting common date ranges since theyre pretty different
start_date = max(fast_food_monthly.index.min(), commodity_monthly.index.min(), meat_production_monthly.index.min())
end_date = min(fast_food_monthly.index.max(), commodity_monthly.index.max(), meat_production_monthly.index.max())

fast_food_monthly = fast_food_monthly.loc[start_date:end_date]
commodity_monthly = commodity_monthly.loc[start_date:end_date]
meat_production_monthly = meat_production_monthly.loc[start_date:end_date]

#merge data
merged_data = pd.concat([fast_food_monthly, commodity_monthly, meat_production_monthly], axis=1)
merged_data = merged_data.dropna()

merged_data = merged_data.replace(',', '', regex=True).astype(float)

#standardize data
scaler = StandardScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(merged_data),
                               columns=merged_data.columns,
                               index=merged_data.index)

#correlation
plt.figure(figsize=(20, 16))
sns.heatmap(normalized_data.corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Heatmap: Fast Food Stocks, Commodities, and Meat Production')
plt.tight_layout()
plt.show()

#would probably want to figure out how to group the meats but imt oo lazy rn
#really good stuff for commodities ig, want more qualitative research. should also really fix these graphs they suck

merged_data.head(10)

In [ ]:
#@title Hasan 3.better: cross correlation

merged_data['Total_Red_Meat'] = merged_data[['Beef_Commercial_Red Meat', 'Beef_Federally Inspected_Red Meat', 'Lamb and Mutton_Commercial_Red Meat', 'Lamb and Mutton_Federally Inspected_Red Meat', 'Pork_Commercial_Red Meat', 'Pork_Federally Inspected_Red Meat', 'Veal_Commercial_Red Meat', 'Veal_Federally Inspected_Red Meat']].sum(axis=1)
merged_data['Total_Poultry'] = merged_data[['Broilers_Federally Inspected_Poultry', 'Other Chicken_Federally Inspected_Poultry', 'Turkey_Federally Inspected_Poultry']].sum(axis=1)
merged_data.drop(['Beef_Commercial_Red Meat', 'Beef_Federally Inspected_Red Meat', 'Lamb and Mutton_Commercial_Red Meat', 'Lamb and Mutton_Federally Inspected_Red Meat', 'Pork_Commercial_Red Meat', 'Pork_Federally Inspected_Red Meat', 'Veal_Commercial_Red Meat', 'Veal_Federally Inspected_Red Meat', 'Broilers_Federally Inspected_Poultry', 'Other Chicken_Federally Inspected_Poultry', 'Turkey_Federally Inspected_Poultry'], axis=1, inplace=True)
merged_data.columns = ["Fast_Food_Stock_Price", "Coffee", "Corn", "Sugar", "Red_Meat", "Poultry"]
scaler = StandardScaler()   #making sure it's standardized ig
normalized_data = pd.DataFrame(scaler.fit_transform(merged_data),
                               columns=merged_data.columns,
                               index=merged_data.index)

def cross_correlation_lags(data, max_lag):
    columns = ['Fast_Food_Stock_Price', 'Coffee', 'Corn', 'Sugar', 'Red_Meat', 'Poultry']
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

#merge data
numeric_data = merged_data[['Fast_Food_Stock_Price', 'Coffee', 'Corn', 'Sugar', 'Red_Meat', 'Poultry']]


max_lag = 24  #its surprisingly good across 2 years
cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

#plot
for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='crest', vmin=-1, vmax=1, center=0, fmt='.2f')
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag} months)')
    plt.tight_layout()
    plt.show()



In [ ]:
#@title Hasan 3.2: causality
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

variables = ['Fast_Food_Stock_Price', 'Coffee', 'Corn', 'Sugar', 'Red_Meat', 'Poultry']
max_lag = 12


granger_results = {}
for cause in variables:
    for effect in variables:
        if cause != effect:
            test_result = grangercausalitytests(merged_data[[cause, effect]], maxlag=max_lag, verbose=False)
            granger_results[(cause, effect)] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]


results_df = pd.DataFrame(index=range(1, max_lag+1), columns=[f"{cause} -> {effect}" for cause in variables for effect in variables if cause != effect])


for (cause, effect), p_values in granger_results.items():
    results_df[f"{cause} -> {effect}"] = p_values
#to make everything numeric and avoid errors
results_df = results_df.apply(pd.to_numeric, errors='coerce')

#color p vals based on significance
def color_p_values(val):
    if pd.isna(val):
        return ''
    elif val <= 0.01:
        return 'background-color: red'
    elif val <= 0.05:
        return 'background-color: green'
    elif val <= 0.1:
        return 'background-color: orange'
    else:
        return ''

styled_results_df = results_df.style.applymap(color_p_values)


print("Granger Causality Test Results (p-values):")
display(styled_results_df)

#results_df.to_csv('granger_causality_results.csv')


In [ ]:
#@title Hasan 3.5 ignore

import numpy as np

corr_matrix = normalized_data.corr()

mask = np.abs(corr_matrix) < 0.5

# Plot the heatmap with the mask
plt.figure(figsize=(20, 16))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, mask=mask)
plt.title('Correlation Heatmap: Fast Food Stocks, Commodities, and Meat Production (Moderate to High Correlations)')
plt.tight_layout()
plt.show()


In [ ]:
#@title Hasan 4.5: commodities vs types of meat regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

commodities = ['Coffee', 'Corn', 'Sugar']
meat_columns = [col for col in normalized_data.columns if 'Meat' in col]

for commodity in commodities:
    for meat_type in meat_columns[:5]:  #analyzing the meat columns
        plt.figure(figsize=(5, 3))
        plt.scatter(normalized_data[commodity], normalized_data[meat_type], alpha=0.5)
        plt.title(f'{commodity} Price vs {meat_type} Production')
        plt.xlabel(f'Normalized {commodity} Price')
        plt.ylabel(f'Normalized {meat_type} Production')
        plt.tight_layout()
        plt.show()

        correlation = normalized_data[[commodity, meat_type]].corr().iloc[0, 1]
        print(f"Correlation between {commodity} price and {meat_type}: {correlation:.4f}")

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

#checks stationarity and differences if necessary
def make_stationary(data):
    for column in data.columns:
        adf_result = adfuller(data[column])
        if adf_result[1] > 0.05:  # p-value > 0.05 indicates non-stationarity
            data[column] = data[column].diff().dropna()
    return data

# VAR models for each commodity-meat pair
for commodity in commodities:
    for meat_type in meat_columns[:5]:
        #prep data here
        data = normalized_data[[commodity, meat_type]].dropna()
        data = make_stationary(data)

        #seeing if there's enough to analyze after making stuff stationary
        if len(data) < 12: # Adjust this threshold as needed
            print(f"Insufficient data for VAR model for {commodity} and {meat_type} after differencing.")
            continue

        # Fit VAR model
        try:
            model = VAR(data)
            results = model.fit(maxlags=12)
        except np.linalg.LinAlgError:
            print(f"LinAlgError encountered for {commodity} and {meat_type}. Possible high multicollinearity or outliers.")
            continue #usually it says that svd did not converge, thinking more about that

        #summaries of var models
        print(f'VAR Model Summary for {commodity} and {meat_type}')
        print(results.summary())

        #impulse-response functions
        irf = results.irf(10)
        irf.plot(orth=False)
        plt.title(f'Impulse Response Function: {commodity} and {meat_type}')
        plt.show()

lot to go through in 4.5. maybe want to find other datasets to avoid multicollinearity

may also want to fit actual lines on the visualization for var regression!

emily kinda want to team up with you on this to see what more work we could do in this area


In [ ]:
#@title Hasan 4.7: commodities and stocks

#time series plot
plt.figure(figsize=(14, 8))
for column in ['Fast_Food_Stock_Price', 'Coffee', 'Corn', 'Sugar']:
    plt.plot(normalized_data.index, normalized_data[column], label=column)
plt.title('Normalized Time Series: Fast Food Stock Price and Commodity Prices')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#@title Hasan 5: meat production, stocks, and unemployment OLD

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import grangercausalitytests

#ACS5YR DATA
acs5yr = import_dataset("1JFpNj9SlUiWwZTFc-pnlggKu6T7h_urP", "acs5yr")
acs5yr = acs5yr.dropna(subset=['Percent'])   #they sabotaged it bro istg
acs5yr = acs5yr[acs5yr['Percent'].str.contains('%')]  #filtering for percentages
acs5yr['Percent'] = acs5yr['Percent'].str.rstrip('%').astype('float') / 100.0   #converting percentages to decimal
acs5yr = acs5yr[acs5yr['Year'].between(2010, 2014)]   #data only available for this years, so everything else is going to have be synced according to this
unemployment = acs5yr.groupby('Year')['Percent'].mean().reset_index()
unemployment['Date'] = pd.to_datetime(unemployment['Year'], format='%Y')
unemployment


#MEAT PRODUCTION DATA
meat_production['Date'] = pd.to_datetime(meat_production['Date'])
meat_production['Year'] = meat_production['Date'].dt.year
meat_production['Production'] = pd.to_numeric(meat_production['Production'], errors='coerce')
meat_production = meat_production.dropna(subset=['Production'])
meat_production = meat_production[meat_production['Year'].between(2010, 2014)]
#aggregate by year and type of meat instead of specific animal
meat_production_agg = meat_production.groupby(['Year', 'Type of Meat'])['Production'].mean().unstack(fill_value=0).reset_index()



# STOCK DATA FOR FAST FOOD RESTAURANTS
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Calculate average stock prices by year
fast_food_stock_prices['Date-Time'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices['Year'] = fast_food_stock_prices['Date-Time'].dt.year
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby('Year')['Close'].mean().reset_index()
avg_fast_food_stock_prices_close.rename(columns={'Close': 'Fast_Food_Stock_Price'}, inplace=True)


#merge dataframes
data = pd.merge(meat_production_agg, unemployment, on='Year', how='inner')
data = pd.merge(data, avg_fast_food_stock_prices_close, on='Year', how='inner')

#ilter for years 2010-2014 since thats whats available for unemployment
data = data[data['Year'].between(2010, 2014)]
data = data.sort_values('Year')
data.columns = ["Year", "Poultry", "Red Meat", "Unemployment_Rate", "Date", "Fast_Food_Stock_Price"]
print("Combined Data:")
print(data)


#CORRELATIONS: good correlations at lag = 0 indicating instantaneous relationships which wouldnt really make sense FUCK

correlation_matrix = data.drop(['Year', 'Date'], axis=1, errors='ignore').corr()
plt.figure(figsize=(6, 5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix Heatmap (Fast Food Stocks)')
plt.tight_layout()
plt.show()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

# Assuming 'data' is your DataFrame
numeric_data = data.drop(['Year', 'Date'], axis=1, errors='ignore') #not sure if we should do that dropping time thing or not

# Define the maximum lag you want to consider
max_lag = 4
# Calculate the cross-correlation matrix for multiple lags
cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

# Plot the cross-correlation matrices for each lag
for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(6, 5))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag})')
    plt.tight_layout()
    plt.show()




#CAUSALITY: only somewhat for poultry -> unemployment

results = {}

#granger causality test for meat production and unemployment rate and vice versa
for meat_type in meat_production_agg.columns[1:]:
    test_result = grangercausalitytests(data[[meat_type, 'Unemployment_Rate']], maxlag = 1, verbose=False)
    results[f'{meat_type} -> Unemployment Rate'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]
    test_result = grangercausalitytests(data[['Unemployment_Rate', meat_type]], maxlag = 1, verbose=False)
    results[f'Unemployment Rate -> {meat_type}'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]

#granger causality test for meat production and fast food stock prices and vice versa
for meat_type in meat_production_agg.columns[1:]:
    test_result = grangercausalitytests(data[[meat_type, 'Fast_Food_Stock_Price']], maxlag = 1, verbose=False)
    results[f'{meat_type} -> Fast Food Stock Price'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]
    test_result = grangercausalitytests(data[['Fast_Food_Stock_Price', meat_type]], maxlag = 1, verbose=False)
    results[f'Fast Food Stock Price -> {meat_type}'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]

#granger causality test for unemployment rate and fast food stock prices and vice versa
test_result = grangercausalitytests(data[['Unemployment_Rate', 'Fast_Food_Stock_Price']], maxlag = 1, verbose=False)
results['Unemployment Rate -> Fast Food Stock Price'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]
test_result = grangercausalitytests(data[['Fast_Food_Stock_Price', 'Unemployment_Rate']], maxlag = 1, verbose=False)
results['Fast Food Stock Price -> Unemployment Rate'] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(1)]

results_df = pd.DataFrame(results, index=[f'Lag {i+1}' for i in range(1)])
print("Granger Causality Test Results (p-values):")
print(results_df)



In [ ]:
#@title Hasan 5.2 better employment data for 5, correlation on meat, commodities, fast food and all stocks, and obesity
from statsmodels.tsa.stattools import grangercausalitytests
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load and process employment data
employment_data = pd.read_csv('employment.csv')
employment_data['Date'] = pd.to_datetime(employment_data[['Year', 'Month']].assign(DAY=1))
employment = employment_data[['Date', 'Employment Rate']]
unemployment = employment_data[['Date', 'Unemployment Rate']]

# Process meat production data
meat_production['Date'] = pd.to_datetime(meat_production['Date'])
meat_production['Production'] = pd.to_numeric(meat_production['Production'], errors='coerce')
meat_production = meat_production.dropna(subset=['Production'])
meat_production_agg = meat_production.groupby(['Date', 'Type of Meat'])['Production'].mean().unstack(fill_value=0).reset_index()

#fast food stock data
all_stocks = pd.read_csv('all_stocks.csv')
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Date'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby(fast_food_stock_prices['Date'].dt.to_period('M'))['Close'].mean().reset_index()
avg_fast_food_stock_prices_close['Date'] = avg_fast_food_stock_prices_close['Date'].dt.to_timestamp()
avg_fast_food_stock_prices_close.rename(columns={'Close': 'Fast_Food_Stock_Price'}, inplace=True)

#all stock data
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])
avg_stock_prices_close = all_stocks.groupby(all_stocks['Date'].dt.to_period('M'))['Close'].mean().reset_index()
avg_stock_prices_close['Date'] = avg_stock_prices_close['Date'].dt.to_timestamp()


#merge data
data = pd.merge(meat_production_agg, unemployment, on='Date', how='inner')
data = pd.merge(data, employment, on='Date', how='inner')
data = pd.merge(data, avg_fast_food_stock_prices_close, on='Date', how='inner')
data = pd.merge(data, avg_stock_prices_close, on='Date', how='inner')


data['Year'] = data['Date'].dt.year
data = data.sort_values('Date')


new_column_names = ["Date", "Poultry", "Red Meat", "Unemployment Rate", "Employment Rate", "Fast_Food_Stock_Price", "Stock_Market", "Year"]
if len(data.columns) == len(new_column_names):
    data.columns = new_column_names
else:
    print("Warning: The number of columns in the DataFrame does not match the expected number.")
    print("Current columns:", data.columns)

print("Combined Data:")
print(data.head())

# CORRELATIONS:
correlation_data = data.drop(['Date'], axis=1)
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix Heatmap (Including Year)')
plt.tight_layout()
plt.show()

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

numeric_data = data.drop(['Date'], axis=1)

max_lag = 12  # Increased to 12 months for a full year of lags

cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag})')
    plt.tight_layout()
    plt.show()


In [ ]:
#@title Hasan 5.3 causality
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

def run_granger_causality(data, max_lag=12):
    variables = ['Poultry', 'Red Meat', 'Employment Rate', 'Unemployment Rate', 'Fast_Food_Stock_Price', 'Stock_Market']
    results = {}

    for i in range(len(variables)):
        for j in range(len(variables)):
            if i != j:
                x = variables[i]
                y = variables[j]
                key = f"{x} -> {y}"
                test_result = grangercausalitytests(data[[x, y]], maxlag=max_lag, verbose=False)
                results[key] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]

    return results

def color_p_values(val):
    if pd.isna(val):
        return ''
    elif val <= 0.01:
        return 'background-color: red'
    elif val <= 0.05:
        return 'background-color: green'
    elif val <= 0.1:
        return 'background-color: orange'
    else:
        return ''

def print_granger_results(results, max_lag):
    results_df = pd.DataFrame(results, index=[f'Lag {i+1}' for i in range(max_lag)]).T
    styled_results_df = results_df.style.applymap(color_p_values)
    display(styled_results_df)

max_lag = 12
granger_results = run_granger_causality(data, max_lag)
print_granger_results(granger_results, max_lag)


In [ ]:
#@title Hasan 5.5 IGNORE
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Assuming the data preparation steps have been completed as previously discussed

# Standardize the data for visualization purposes
scaler = StandardScaler()
data_standardized = pd.DataFrame(scaler.fit_transform(data[['Poultry', 'Red Meat', 'Fast_Food_Stock_Price']]),
                                 columns=['Poultry', 'Red Meat', 'Fast_Food_Stock_Price'],
                                 index=data.index)
data_standardized['Year'] = data['Year']
data_standardized['Unemployment Rate'] = data['Unemployment Rate']  # Keep unemployment rate as percentage

# Create the plot
fig, ax1 = plt.subplots(figsize=(14, 8))

# Plot standardized variables
ax1.plot(data_standardized['Year'], data_standardized['Poultry'], label='Poultry Production', color='blue')
ax1.plot(data_standardized['Year'], data_standardized['Red Meat'], label='Red Meat Production', color='red')
ax1.plot(data_standardized['Year'], data_standardized['Fast_Food_Stock_Price'], label='Fast Food Stock Price', color='green')

# Set labels for standardized axis
ax1.set_xlabel('Year')
ax1.set_ylabel('Standardized Values')

# Create a secondary y-axis for unemployment rate
ax2 = ax1.twinx()
ax2.plot(data['Year'], data['Unemployment Rate'], label='Unemployment Rate', color='purple', linestyle='--')
ax2.set_ylabel('Unemployment Rate (%)')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

# Set title and display the plot
plt.title('Meat Production, Unemployment Rate, and Fast Food Stock Prices (2010-2014)')
plt.tight_layout()
plt.show()


In [ ]:
#@title Hasan 5.7 graphing food stocks, unemployment, meat production, and stock market

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is your DataFrame with columns 'Date', 'Poultry', 'Red Meat', 'Unemployment Rate', 'Employment Rate', 'Fast_Food_Stock_Price', 'Stock_Market'
data['Date'] = pd.to_datetime(data['Date'])

# Standardize the data
scaler = StandardScaler()
columns_to_standardize = ['Poultry', 'Red Meat', 'Unemployment Rate', 'Employment Rate', 'Fast_Food_Stock_Price', 'Stock_Market']
data_standardized = pd.DataFrame(scaler.fit_transform(data[columns_to_standardize]), columns=columns_to_standardize, index=data.index)
data_standardized['Date'] = data['Date']

# Create the plot
fig, ax1 = plt.subplots(figsize=(18, 9))

# Plot Poultry and Red Meat on the first y-axis
ax1.plot(data_standardized['Date'], data_standardized['Poultry'], label='Poultry', color='blue')
ax1.plot(data_standardized['Date'], data_standardized['Red Meat'], label='Red Meat', color='red')
ax1.set_xlabel('Date')
ax1.set_ylabel('Standardized Meat Production', color='black')
ax1.tick_params(axis='y', labelcolor='black')

# Create a second y-axis for Unemployment Rate and Employment Rate
ax2 = ax1.twinx()
ax2.plot(data_standardized['Date'], data_standardized['Unemployment Rate'], label='Unemployment Rate', color='green')
ax2.plot(data_standardized['Date'], data_standardized['Employment Rate'], label='Employment Rate', color='orange')
ax2.set_ylabel('Standardized Rates (%)', color='green')
ax2.tick_params(axis='y', labelcolor='green')

# Create a third y-axis for Fast Food Stock Price and Stock Market
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('axes', 1.1))  # Offset the right spine of the third axis
ax3.plot(data_standardized['Date'], data_standardized['Fast_Food_Stock_Price'], label='Fast Food Stock Price', color='purple')
ax3.plot(data_standardized['Date'], data_standardized['Stock_Market'], label='Stock Market', color='brown')
ax3.set_ylabel('Standardized Stock Prices ($)', color='purple')
ax3.tick_params(axis='y', labelcolor='purple')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(lines1 + lines2 + lines3, labels1 + labels2 + labels3, loc='upper left')

plt.title('Standardized Time Series of Meat Production, Employment, Unemployment, Fast Food Stocks, and Stock Market')
plt.tight_layout()
plt.show()


In [ ]:
#@title Hasan 6 finding more correlations and causation for obestiy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler

# Load and process employment data
employment_data = pd.read_csv('employment.csv')
employment_data['Year'] = pd.to_datetime(employment_data['Date']).dt.year
employment = employment_data.groupby('Year')['Employment Rate'].mean().reset_index()
unemployment = employment_data.groupby('Year')['Unemployment Rate'].mean().reset_index()

# Process meat production data
meat_production = pd.read_csv('meat_stats_meat_production.csv')
meat_production['Year'] = pd.to_datetime(meat_production['Date']).dt.year
meat_production['Production'] = pd.to_numeric(meat_production['Production'], errors='coerce')
meat_production = meat_production.dropna(subset=['Production'])
meat_production_agg = meat_production.groupby(['Year', 'Type of Meat'])['Production'].mean().unstack(fill_value=0).reset_index()

# Fast food stock data
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Year'] = pd.to_datetime(fast_food_stock_prices['Date-Time']).dt.year
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby('Year')['Close'].mean().reset_index()
avg_fast_food_stock_prices_close.rename(columns={'Close': 'Fast_Food_Stock_Price'}, inplace=True)

# All stock data
all_stocks['Year'] = pd.to_datetime(all_stocks['Date-Time']).dt.year
avg_stock_prices_close = all_stocks.groupby('Year')['Close'].mean().reset_index()
avg_stock_prices_close.rename(columns={'Close': 'Stock_Market'}, inplace=True)

# Load and process obesity data
nutrition = pd.read_csv('nutrition.csv')
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]
obesity_by_year = obesity_data.groupby('YearStart')['Data_Value'].mean().reset_index()
obesity_by_year.columns = ['Year', 'ObesityRate']

# Merge data
data = pd.merge(meat_production_agg, unemployment, on='Year', how='inner')
data = pd.merge(data, employment, on='Year', how='inner')
data = pd.merge(data, avg_fast_food_stock_prices_close, on='Year', how='inner')
data = pd.merge(data, avg_stock_prices_close, on='Year', how='inner')
data = pd.merge(data, obesity_by_year, on='Year', how='inner')

data = data.sort_values('Year')

print("Combined Data:")
print(data.head())

# CORRELATIONS:
correlation_data = data.drop(['Year'], axis=1)
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

numeric_data = data.drop(['Year'], axis=1)

max_lag = 5

cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(12, 10))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag} years)')
    plt.tight_layout()
    plt.show()




In [ ]:
#@title Hasan 6.2 causations for obesity!! help!!

import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from IPython.display import display

# Assuming 'data' is your DataFrame with yearly data

variables = ['ObesityRate', 'Poultry', 'Red Meat', 'Unemployment Rate', 'Employment Rate', 'Fast_Food_Stock_Price', 'Stock_Market']
max_lag = 2  # Adjusted for yearly data

granger_results = {}
for cause in variables:
    for effect in variables:
        if cause != effect:
            test_result = grangercausalitytests(data[[cause, effect]], maxlag=max_lag, verbose=False)
            granger_results[(cause, effect)] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]

results_df = pd.DataFrame(index=range(1, max_lag+1), columns=[f"{cause} -> {effect}" for cause in variables for effect in variables if cause != effect])

for (cause, effect), p_values in granger_results.items():
    results_df[f"{cause} -> {effect}"] = p_values

# Convert to numeric values
results_df = results_df.apply(pd.to_numeric, errors='coerce')

# Color p-values based on significance
def color_p_values(val):
    if pd.isna(val):
        return ''
    elif val <= 0.01:
        return 'background-color: red'
    elif val <= 0.05:
        return 'background-color: green'
    elif val <= 0.1:
        return 'background-color: orange'
    else:
        return ''

styled_results_df = results_df.style.applymap(color_p_values)

print("Granger Causality Test Results (p-values):")
display(styled_results_df)

# Create a new DataFrame with only obesity-related results
obesity_results = results_df[[col for col in results_df.columns if 'ObesityRate' in col]]

# Style the obesity results
styled_obesity_results = obesity_results.style.applymap(color_p_values)

print("\nGranger Causality Test Results for Obesity Rate (p-values):")
display(styled_obesity_results)

# Optionally, save the results to CSV files
# results_df.to_csv('granger_causality_results.csv')
# obesity_results.to_csv('obesity_granger_causality_results.csv')



In [ ]:
#@title 7 cold storage?!?!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler

# Load and process employment data
employment_data = pd.read_csv('employment.csv')
employment_data['Year'] = pd.to_datetime(employment_data['Date']).dt.year
employment = employment_data.groupby('Year')['Employment Rate'].mean().reset_index()
unemployment = employment_data.groupby('Year')['Unemployment Rate'].mean().reset_index()

# Process meat production data
meat_production = pd.read_csv('meat_stats_meat_production.csv')
meat_production['Year'] = pd.to_datetime(meat_production['Date']).dt.year
meat_production['Production'] = pd.to_numeric(meat_production['Production'], errors='coerce')
meat_production = meat_production.dropna(subset=['Production'])
meat_production_agg = meat_production.groupby(['Year', 'Type of Meat'])['Production'].mean().unstack(fill_value=0).reset_index()

# Fast food stock data
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Year'] = pd.to_datetime(fast_food_stock_prices['Date-Time']).dt.year
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby('Year')['Close'].mean().reset_index()
avg_fast_food_stock_prices_close.rename(columns={'Close': 'Fast_Food_Stock_Price'}, inplace=True)

# All stock data
all_stocks['Year'] = pd.to_datetime(all_stocks['Date-Time']).dt.year
avg_stock_prices_close = all_stocks.groupby('Year')['Close'].mean().reset_index()
avg_stock_prices_close.rename(columns={'Close': 'Stock_Market'}, inplace=True)

# Load and process obesity data
nutrition = pd.read_csv('nutrition.csv')
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]
obesity_by_year = obesity_data.groupby('YearStart')['Data_Value'].mean().reset_index()
obesity_by_year.columns = ['Year', 'ObesityRate']

# Merge data
data = pd.merge(meat_production_agg, unemployment, on='Year', how='inner')
data = pd.merge(data, employment, on='Year', how='inner')
data = pd.merge(data, avg_fast_food_stock_prices_close, on='Year', how='inner')
data = pd.merge(data, avg_stock_prices_close, on='Year', how='inner')
data = pd.merge(data, obesity_by_year, on='Year', how='inner')

# Process and aggregate cold storage data
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])
meat_stats_cold_storage['Year'] = meat_stats_cold_storage['Date'].dt.year
cold_storage_agg = meat_stats_cold_storage.groupby(['Year', 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0).reset_index()
cold_storage_agg.columns = ['Year', 'ColdStorageRedMeat', 'ColdStoragePoultry']

# Merge cold storage data
data = pd.merge(data, cold_storage_agg, on='Year', how='inner')

print("Combined Data with Cold Storage:")
print(data.head())

# CORRELATIONS:
correlation_data = data.drop(['Year'], axis=1)
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

numeric_data = data.drop(['Year'], axis=1)

max_lag = 5

cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(12, 10))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag} years)')
    plt.tight_layout()
    plt.show()

max_lag = 2
granger_results = {}
for cause in numeric_data.columns:
    for effect in numeric_data.columns:
        if cause != effect:
            try:
                test_result = grangercausalitytests(data[[cause, effect]], maxlag=max_lag, verbose=False)
                granger_results[(cause, effect)] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]
            except Exception as e:
                print(f"Error testing {cause} -> {effect}: {e}")
                granger_results[(cause, effect)] = [np.nan] * max_lag

results_df = pd.DataFrame(index=range(1, max_lag+1), columns=[f"{cause} -> {effect}" for cause in numeric_data.columns for effect in numeric_data.columns if cause != effect])

for (cause, effect), p_values in granger_results.items():
    results_df[f"{cause} -> {effect}"] = p_values

# Convert to numeric values
results_df = results_df.apply(pd.to_numeric, errors='coerce')

# Color p-values based on significance
def color_p_values(val):
    if pd.isna(val):
        return ''
    elif val <= 0.01:
        return 'background-color: red'
    elif val <= 0.05:
        return 'background-color: green'
    elif val <= 0.1:
        return 'background-color: orange'
    else:
        return ''

styled_results_df = results_df.style.applymap(color_p_values)

print("Granger Causality Test Results (p-values):")
display(styled_results_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import StandardScaler

#reload data to exclude obesity
employment_data = pd.read_csv('employment.csv')
employment_data['Date'] = pd.to_datetime(employment_data['Date'])
employment_data['YearMonth'] = employment_data['Date'].dt.to_period('M')
employment = employment_data.groupby('YearMonth')['Employment Rate'].mean().reset_index()
unemployment = employment_data.groupby('YearMonth')['Unemployment Rate'].mean().reset_index()

#meat
meat_production = pd.read_csv('meat_stats_meat_production.csv')
meat_production['Date'] = pd.to_datetime(meat_production['Date'])
meat_production['YearMonth'] = meat_production['Date'].dt.to_period('M')
meat_production['Production'] = pd.to_numeric(meat_production['Production'], errors='coerce')
meat_production = meat_production.dropna(subset=['Production'])
meat_production_agg = meat_production.groupby(['YearMonth', 'Type of Meat'])['Production'].mean().unstack(fill_value=0).reset_index()

#fast food stocks
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Date'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices['YearMonth'] = fast_food_stock_prices['Date'].dt.to_period('M')
avg_fast_food_stock_prices_close = fast_food_stock_prices.groupby('YearMonth')['Close'].mean().reset_index()
avg_fast_food_stock_prices_close.rename(columns={'Close': 'Fast_Food_Stock_Price'}, inplace=True)

#all stocks
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])
all_stocks['YearMonth'] = all_stocks['Date'].dt.to_period('M')
avg_stock_prices_close = all_stocks.groupby('YearMonth')['Close'].mean().reset_index()
avg_stock_prices_close.rename(columns={'Close': 'Stock_Market'}, inplace=True)

#merge data
data = pd.merge(meat_production_agg, unemployment, on='YearMonth', how='inner')
data = pd.merge(data, employment, on='YearMonth', how='inner')
data = pd.merge(data, avg_fast_food_stock_prices_close, on='YearMonth', how='inner')
data = pd.merge(data, avg_stock_prices_close, on='YearMonth', how='inner')

# aggregate by month now since obesity's excluded
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])
meat_stats_cold_storage['YearMonth'] = meat_stats_cold_storage['Date'].dt.to_period('M')
cold_storage_agg = meat_stats_cold_storage.groupby(['YearMonth', 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0).reset_index()
cold_storage_agg.columns = ['YearMonth', 'ColdStorageRedMeat', 'ColdStoragePoultry']

#merge
data = pd.merge(data, cold_storage_agg, on='YearMonth', how='inner')

print("Combined Data with Cold Storage (Monthly):")
print(data.head())

# CORRELATIONS:
correlation_data = data.drop(['YearMonth'], axis=1)
correlation_matrix = correlation_data.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix Heatmap')
plt.tight_layout()
plt.show()

def cross_correlation_lags(data, max_lag):
    columns = data.columns
    cross_corr_lags = {lag: pd.DataFrame(index=columns, columns=columns) for lag in range(-max_lag, max_lag + 1)}

    for i in columns:
        for j in columns:
            cross_corr = sm.tsa.stattools.ccf(data[i], data[j], adjusted=False)
            for lag in range(-max_lag, max_lag + 1):
                if lag < 0:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[-lag]
                else:
                    cross_corr_lags[lag].loc[i, j] = cross_corr[lag]

    return cross_corr_lags

numeric_data = data.drop(['YearMonth'], axis=1)

max_lag = 5

cross_corr_lags = cross_correlation_lags(numeric_data, max_lag)

for lag in range(-max_lag, max_lag + 1):
    plt.figure(figsize=(12, 10))
    sns.heatmap(cross_corr_lags[lag].astype(float), annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Cross-Correlation Matrix Heatmap (Lag {lag} months)')
    plt.tight_layout()
    plt.show()

granger_results = {}
for cause in numeric_data.columns:
    for effect in numeric_data.columns:
        if cause != effect:
            try:
                test_result = grangercausalitytests(data[[cause, effect]], maxlag=max_lag, verbose=False)
                granger_results[(cause, effect)] = [test_result[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]
            except Exception as e:
                print(f"Error testing {cause} -> {effect}: {e}")
                granger_results[(cause, effect)] = [np.nan] * max_lag

results_df = pd.DataFrame(index=range(1, max_lag+1), columns=[f"{cause} -> {effect}" for cause in numeric_data.columns for effect in numeric_data.columns if cause != effect])

for (cause, effect), p_values in granger_results.items():
    results_df[f"{cause} -> {effect}"] = p_values

# Convert to numeric values
results_df = results_df.apply(pd.to_numeric, errors='coerce')

# Color p-values based on significance
def color_p_values(val):
    if pd.isna(val):
        return ''
    elif val <= 0.01:
        return 'background-color: red'
    elif val <= 0.05:
        return 'background-color: green'
    elif val <= 0.1:
        return 'background-color: orange'
    else:
        return ''

styled_results_df = results_df.style.applymap(color_p_values)

print("Granger Causality Test Results (p-values):")
display(styled_results_df)


# Filter results to include only columns that involve ColdStorageRedMeat or ColdStoragePoultry
cold_storage_columns = [col for col in results_df.columns if 'ColdStorageRedMeat' in col or 'ColdStoragePoultry' in col]
filtered_results_df = results_df[cold_storage_columns]

# Style the filtered results
styled_filtered_results_df = filtered_results_df.style.applymap(color_p_values)

print("Granger Causality Test Results Involving Cold Storage Data (p-values):")
display(styled_filtered_results_df)

In [ ]:
#@title Hasan table
import pandas as pd
from tabulate import tabulate

# Create a dictionary for the data
data = {
    #'No.': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    'Variable X': [
        'Fast Food Stocks', 'Fast Food Consumption', 'Unemployment Rate',
        'Unemployment Rate', 'Obesity Rates', 'Obesity','Obesity','aaCold Storage Red Meat Levels (CSRM)',
        'Red Meat Production', 'Red Meat Production', 'Cold Storage Poultry (CSP)',
        'Cold Storage Poultry (CSP)', 'Cold Storage Poultry (CSP)', 'Cold Storage Poultry (CSP)',
        'Cold Storage Poultry (CSP)', 'Poultry Production', 'Red Meat Production',
        'Red Meat Production', 'Unemployment Rate', 'Employment Rate', 'Fast Food Stock Prices (FFSP)'
    ],
    'Variable Y': [
        'Obesity Rate', 'Poultry Production', 'Stock Market Performance',
        'Fast Food Stock Prices', 'Red Meat Consumption','Fast Food Stock Prices','Market Stock Prices', 'aaRed Meat Production',
        'Employment Levels', 'Economic Performance', 'Red Meat Production',
        'Employment Levels', 'Fast Food Stock Prices', 'Stock Market Performance',
        'Obesity Rates', 'Cold Storage Meat (CSM)', 'Cold Storage Red Meat Levels (CSRM)',
        'Cold Storage Poultry (CSP)', 'Cold Storage Red Meat Levels (CSRM)',
        'Cold Storage Poultry', 'Cold Storage Poultry (CSP)'
    ],
    'Correlation': [
        0.84, 0.81, -0.98, -0.98, -0.85, 0.80 ,0.75, 'aa-0.63 to -0.72', 'N/A', 'N/A', 'N/A',
        'N/A', 0.54, 0.59, 0.57, 'N/A', 'N/A', 'N/A', 'N/A', '',''
    ],
    'X to Y Causation': [
        'No', 'Yes (p=0.01)', 'No', 'Yes (p=0.05)', 'No', 'aaYes (p=0.01)', 'Yes (p=0.01)',
        'Yes (p=0.01)', 'Yes (p=0.05)', 'Yes (p=0.1)', 'Yes (p=0.01)', 'Yes (p=0.05)',
        'Yes (p=0.05)', 'Yes (p=0.01)', 'Yes (p=0.01)', 'Yes (p=0.1)', 'Yes (p=0.05)',
        'Yes (p=0.1)', 'Yes (p=0.1)','',''
    ],
    'Y to X Causation': [
        'No', 'Yes (p=0.01)', 'No', 'Yes (p=0.05)', 'No', 'aaYes (p=0.01)', 'Yes (p=0.05)',
        'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No','',''
    ],
    'Lags in discussion': [
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,'',''
    ],
    'Explanation': [
        'Parallel but not predictive increase', 'Chicken is the number 1 item used in fast food', 'Lower unemployment rates are associated with higher stock market performance (Philips Curve)',
        'Lower unemployment rates are associated with higher fast food stock performance (Philips Curve)', 'Red meat consumption might be associated with lower obesity rates, though this relationship is complex and may involve other factors',
        'aa', '', '', '', '', '', '', '', '', '', '', '', '', '','',''
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the table
print(tabulate(df, headers='keys', tablefmt='fancy'))

In [ ]:
#@title Ikenna's code
# this analysis is time-independent -> Illinois has highest and Virgin Islands has lowest
# Filter the dataset for physical activity data
pa_data = nutrition[nutrition['Class'] == 'Physical Activity'].head(300)

# Bar Plot of Physical Activity Levels
plt.figure(figsize=(14, 7))
pa_state_means = pa_data.groupby('LocationDesc')['Data_Value'].mean().sort_values()
pa_state_means.plot(kind='bar', color='skyblue')
plt.title('Average Percentage of Students Achieving Physical Activity Recommendations by State')
plt.xlabel('State')
plt.ylabel('Percentage')
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#@title Ikenna 2
# Looking at the visualization for children's nutrition, consumption, and physical activity over time
# group by starting year
n_df = nutrition.dropna()
sugar_drinks_df = n_df[n_df["Class"]== "Sugar Drinks"]
fruits_veg_df = n_df[n_df["Class"]== "Fruits and Vegetables"]
physical_activity_df = n_df[n_df["Class"]== "Physical Activity"]
obesity_df = n_df[n_df["Class"]== "Obesity / Weight Status"]
tv_df = n_df[n_df["Class"]== "Television Viewing"]

new_sugar_drinks_df = sugar_drinks_df.groupby("YearStart")["Data_Value"].mean()
new_fruits_veg_df = fruits_veg_df.groupby("YearStart")["Data_Value"].mean()
new_physical_activity_df = physical_activity_df.groupby("YearStart")["Data_Value"].mean()
new_obesity_df = obesity_df.groupby("YearStart")["Data_Value"].mean()
new_tv_df = tv_df.groupby("YearStart")["Data_Value"].mean()


# Plot the trends
plt.figure(figsize=(14, 7))
plt.plot(new_fruits_veg_df.index, new_fruits_veg_df.values, marker='o', linestyle='-', color='green', label='fruits and vegetables')
plt.plot(new_sugar_drinks_df.index, new_sugar_drinks_df.values, marker='o', linestyle='-', color='orange', label='sugar drinks')
plt.plot(new_physical_activity_df.index, new_physical_activity_df.values, marker='o', linestyle='-', color='blue', label='physical activity')
plt.plot(new_obesity_df.index, new_obesity_df.values, marker='o', linestyle='-', color='red', label='obesity')
plt.plot(new_tv_df.index, new_tv_df.values, marker='o', linestyle='-', color='purple', label='tv')

plt.title("Average Percent of students in grades 9-12 who drank regular soda/pop at least one time per day")
plt.xlabel("Year")
plt.ylabel("Percent Consumed")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Ikenna's Code: Data Wrangling to produce health-region csv

In [ ]:
# 1: preparing the CDI data

cdi_file = "/content/drive/MyDrive/Datathon Practice/US-CDI-Report.csv" # can replace
cdi_data = pd.read_csv(cdi_file)
dropped_cdi_columns = ["DataSource", "Response", "DataValueFootnoteSymbol", "DatavalueFootnote", "StratificationCategory1", "Stratification1","StratificationCategory2", "Stratification2", "StratificationCategory3", "Stratification3", "ResponseID", "DataValueTypeID","StratificationCategoryID2", "StratificationID2", "StratificationCategoryID3", "StratificationID3"]
reduced_cdi_data = cdi_data.drop(columns=dropped_cdi_columns)
included_topics = ['Diabetes', 'Cancer','Nutrition, Physical Activity, and Weight Status']
# drop entries with no value for the DataValue
reduced2_cdi_data = reduced_cdi_data[reduced_cdi_data["Topic"].isin(included_topics)].dropna(subset=["DataValue"])

# Currently ignoring the non-hispanic discrepancy along ethnicity to focus on race

# Checking the composition of different topics in the dataset
condition = reduced2_cdi_data["Topic"] == "Diabetes"
d_count = condition.sum()
condition2 = reduced2_cdi_data["Topic"] == "Nutrition, Physical Activity, and Weight Status"
n_count = condition2.sum()
condition3 = reduced2_cdi_data["Topic"] == "Cancer"
c_count = condition3.sum()
print(f"Diabetes Entry Count: {d_count}. Diabetes Percentage: {d_count/total_count*100}%")
print(f"Nutrition Entry Count: {n_count}. Nutrition Percentage: {n_count/total_count*100}%")
print(f"Cancer Entry Count: {c_count}. Cancer Percentage: {c_count/total_count*100}%")

# rename categorical symbols to match nutrition dataset
race_name_change = {
    'GENM': 'MALE',
    'GENF': 'FEMALE',
    'AIAN': 'RACENAA',
    'ASN': 'RACEASN',
    'WHT': 'RACEWHT',
    'BLK': 'RACEBLK',
    'OVR': 'OVERALL',
    'API': 'RACEHPI',
    'APIO': 'RACEHPI',
    'AIAO': 'RACENAA',
    'MRC': 'RACE2PLUS',
    'OTH': 'RACEOTH'
}

reduced2_cdi_data['StratificationID1'] = reduced2_cdi_data['StratificationID1'].replace(race_name_change)

# us changes: renaming + change Number for DataValueType to Value,
reduced2_cdi_data['DataValueType'] = reduced2_cdi_data['DataValueType'].replace({'Number': 'Value'})
# make sample size value -1 to denote no sample size provided
cdi_column_renaming = {
    "DataValue":"Data_Value",
    "DataValueAlt":"Data_Value_Alt",
    "LowConfidenceLimit":"Low_Confidence_Limit",
    "HighConfidenceLimit":"High_Confidence_Limit",
    "StratificationCategoryID1":"StratificationCategoryId1",
}

reduced2_cdi_data.rename(columns=cdi_column_renaming, inplace=True)

# since sample sizes are provided in the nutrition dataset, make sample size value -1 to denote no sample size provided.
reduced2_cdi_data["Sample_Size"] = "-1"

import re
# switch coordinate from Point to tuple for potential location analysis with longitude-latitude coordinates
def location_parsing_cdi(s):
    if pd.isna(s):
        return None
    pattern = re.compile(r'[()]| ')
    splits = re.split(pattern, s)
    t = (float(splits[2]), float(splits[3]))
    return t
reduced2_cdi_data["GeoLocation"] = reduced2_cdi_data["GeoLocation"].apply(location_parsing_cdi)

# get the types for the GeoLocation to ensure that there are only tuple types
def get_column_types(column):
    return set(column.apply(lambda entry: type(entry)))
vals = get_column_types(reduced2_cdi_data["GeoLocation"])

# 2: Prepare the nutrition dataset
nutrition_data = "/content/drive/MyDrive/Datathon Practice/Nutrition_Physical_Activity_and_Obesity_Data.csv" # can replace
nutrition_df = pd.read_csv(nutrition_data)

# drop uneeded columns
dropped_nutrition_columns = ["Datasource", "Data_Value_Unit", "Data_Value_Type", "Topic", "Data_Value_Footnote_Symbol","Data_Value_Footnote","Total","Age(years)","Education", "Gender", "Grade", "Income", "Race/Ethnicity", "DataValueTypeID", "StratificationCategory1", "Stratification1", "ClassID"]
reduced_nutrition_df = nutrition_df.drop(columns=dropped_nutrition_columns)
# Rename class to topic
reduced_nutrition_df.rename(columns={'Class': 'Topic'}, inplace=True)

# parse the nutrition datset for the longitude and latitude values
def location_parsing_nutrition(s):
    if pd.isna(s):
        return None
    pattern = re.compile(r'[()]|(?:, )')
    splits = re.split(pattern, s)
    t = (float(splits[1]), float(splits[2]))
    return t
reduced_nutrition_df["GeoLocation"] = reduced_nutrition_df["GeoLocation"].apply(location_parsing_nutrition)

n_vals = get_column_types(reduced_nutrition_df["GeoLocation"])
print(n_vals)

# combine the dataframes
merged_df = pd.concat([reduced_nutrition_df, reduced2_cdi_data])
# reset the indices
merged_df.reset_index(drop=True, inplace=True)

# 3: save the merged dataset as a csv
merged_df.to_csv('/content/drive/MyDrive/Datathon Practice/health-region.csv', index=False)

# ensure that the merged dataset has the desired features
merged_df["Topic"].unique()


Ikenna's Code: Producing Regional Graphs

In [ ]:
import pandas as pd
import plotly.express as px

# rename health_region_df to whatever the health-region.csv DF is called
diabetes_df = health_region_df[health_region_df["Topic"]=="Diabetes"]
isNotTargetYear = diabetes_df["YearStart"] >= 2011
diabetes_target_years = diabetes_df[isNotTargetYear].reset_index()
plt.hist(diabetes_target_years["YearStart"])

# get the topic questions for the pipeline
print(diabetes_target_years["Question"].unique())

# make this a function
def getCombo(df):
  units = df["DataValueUnit"].unique()
  types = df["DataValueType"].unique()
  max_count = 0
  final_combo = ["",""]
  dUnit = ""
  dType = ""
  for i in units:
    dUnit = i
    # check for none type: (should probably deal with this early when cleaning the data)
    if i:
      # get a df with all entries of that unit
      unit_temp = df[df["DataValueUnit"]==i]
      # get the unit-type combo with the max entries
      for j in types:
        type_temp = unit_temp[unit_temp["DataValueType"]==j]
        # we'll prioritize age-adjusted over anything else
        if j=="Age-adjusted Rate" and len(type_temp) > 0:
          final_combo = [i,j]
          return final_combo
        if len(type_temp) > max_count:
          max_count = len(type_temp)
          # make this unit-type combo the one to track, only changes when max_count changes
          final_combo = [i, j]
  return final_combo

def qSubset(df, questions, case_sensitive=False):
    """
    Filter the DataFrame based on a list of questions.

    Parameters:
    df (pd.DataFrame): The DataFrame to filter.
    questions (list): A list of question strings to filter by.
    case_sensitive (bool): If True, perform case-sensitive filtering. Default is False.

    Returns:
    pd.DataFrame: A DataFrame containing only the rows with questions in the provided list.
    """
    if not case_sensitive:
        questions = [q.lower() for q in questions]
        output = df[df["Question"].str.lower().isin(questions)]
    else:
        output = df[df["Question"].isin(questions)]
    return output

def getGraph(df, question, combo, topic):
  # Filter the DataFrame for questions related to diabetes mortality (only 5219 data points for mortality :/)
  subset = df[(df["DataValueUnit"]==combo[0]) & (df["DataValueType"]==combo[1])].reset_index()

  # Convert 'Data_Value' column to numeric, handling non-numeric values
  subset['Data_Value'] = pd.to_numeric(subset['Data_Value'], errors='coerce')

  # Group by state and calculate the average Data_Value
  state_averages = subset.groupby('LocationAbbr')['Data_Value'].mean().reset_index()

  # Create a choropleth map
  fig = px.choropleth(
      state_averages,
      locations='LocationAbbr',
      locationmode="USA-states",
      color='Data_Value',
      color_continuous_scale=px.colors.sequential.Tealgrn,  # Optional: choose a color scale
      scope="usa",
      title=topic
  )

  # Display the map
  fig.show()

### Look at diabetes mortality
def regionDiabetesPipeline(topic_qs, topic):
  subset = qSubset(diabetes_target_years, topic_qs)
  combo = getCombo(subset)
  getGraph(subset, topic_qs, combo, topic)
  # f"{topic} from 2011-2022. {combo[0]}, {combo[1]}" initially printed out unit metrics to determine the best metrics
mortality_qs = ['Mortality due to diabetes reported as any listed cause of death','Mortality with diabetic ketoacidosis reported as any listed cause of death', ]
regionDiabetesPipeline(mortality_qs, "Age-Adjusted Rate of Cases per 100,000 of Mortality due to Diabetes as any Listed Cause of Death from 2011 to 2022")

morbidity_qs = ['Amputation of a lower extremity attributable to diabetes']
regionDiabetesPipeline(morbidity_qs, "Age-Adjusted Rate of Cases per 10,000 of Amputations of a Lower Extremity Attributable to Diabetes from 2011 to 2022")

hospitalization_qs = ['Hospitalization with diabetes as a listed diagnosis']
regionDiabetesPipeline(hospitalization_qs, "Age-Adjusted Rate of Cases per 10,000 of Hospitalizations with Diabetes as a Listed Diagnosis from 2011 to 2022")

bloodPressure_qs = ['Prevalence of high blood pressure among adults aged >= 18 years with diagnosed diabetes']
regionDiabetesPipeline(bloodPressure_qs, "Age-Adjusted Rate of the Percentage of High Blood Pressure Prevalence Amongst Adults 18 years or Older with Diagnosed Diabetes")

In [ ]:
#@title Oliver's code

#@title Importing libraries
# Run if it gives you an error for any of these; FEEL FREE TO IMPORT MORE FOR EVERYONE TO USE!


# Data manipulation and analysis
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns

# Data visualization
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine learning
import sklearn
import tensorflow as tf
import torch

from sklearn import preprocessing

# Statistical analysis
# import statsmodels.api as sm

# Natural Language Processing
# import nltk
# import spacy

# Geospatial analysis
# import geopandas as gpd
# import folium



In [ ]:
#@title Import Proxy Loan Dataset
loans0 = import_dataset("1LUw8MtYYFNapLUmt-ISqk_YzglegoYut", "1991-1999")
loans1 = import_dataset("1Z1KjplSitwZaHU4bJrJMdFLTJBdXgxI7", "2000-2009")
loans2 = import_dataset("1uhW_2RkHtHXQxq4TpYq5UlcTg-4hrOvY", "2010-2019")
loans3 = import_dataset("11RhgGiwaiXnROI4ZBxIHfetVK_03urzz", "2010-present")
loans0 = loans0[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans1 = loans1[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans2 = loans2[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]
loans3 = loans3[['BorrName', 'BorrZip', 'ProjectCounty', "NaicsCode", 'NaicsDescription', 'FranchiseCode', 'FranchiseName', 'ApprovalFiscalYear', 'ApprovalDate']]

chain_names = ['POPEYE', 'MCDONALD', 'chick-fil-a', 'TACO BELL', 'DAIRY QUEEN', 'WAFFLE HOUSE', 'FRIED', 'KENTUCKY', 'BURGER', 'pizza', 'Arctic Circle', 'Chili', 'sonic drive', 'jack in the box', 'panda express', 'pei wei','papa john', 'little caesar', 'wendy\'s', 'wingstop', 'zaxby', 'jimmy john', 'five guys', 'hardee', 'bojangle', 'carl\'s jr', 'dunkin', 'krispy kreme', 'el pollo loco', 'shake shack', 'baskin robbins', 'church\'s chicken', 'papa murphy\'s', 'moe\s', 'freddy\'s frozen']
chain_regex = '|'.join(chain_names)
grocery_indicators = ['Grocery', 'Food store', 'bodega', 'food mart']
grocery_regex = '|'.join(grocery_indicators)

In [ ]:
#@title Pipeline for Getting Data (Example here based on Fiscal Year)
for i in range(4):
    var_name = f'loans{i}'
    result_name = f'chains{i}'
    globals()[result_name] = globals()[var_name][globals()[var_name]['BorrName'].str.contains(chain_regex, case=False, na=False)]
    print(globals()[result_name]['ApprovalFiscalYear'].value_counts(dropna=False))

In [ ]:
#@title Multivariate Regressions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
#@title Testing correlations

clean = import_dataset('1Pi2g3ep0xoGuxJbqt6iHqYWoc3PRgOOq', 'clean')
plt.scatter(np.array(clean['PCT_FREE_LUNCH15']), np.array(clean['Data_Value']))
correlation = clean['PCT_FREE_LUNCH15'].corr(clean['Data_Value'])
print("Correlation between SNAP and Diabetes:", correlation)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X = clean[['PCT_FREE_LUNCH15', 'PCT_LACCESS_SNAP15', 'FFRPTH16', 'CONVSPTH16', 'PC_FFRSALES12']]
y = clean['Data_Value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)

# Evaluation
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_rf = mean_squared_error(y_test, y_pred_rf)

print("Linear Regression MSE:", mse_lr)
print("Random Forest MSE:", mse_rf)

# Goal is to stay as close to the diagonal as possible
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_lr, color='red', label='LR Predictions', alpha=0.5)
plt.scatter(y_test, y_pred_rf, color='blue', label='RF Predictions', alpha=0.5)
plt.title('Actual vs. Predicted Values')
plt.xlabel('Actual Indicator')
plt.ylabel('Predicted Indicator')
plt.legend()
plt.show()

In [ ]:
# Creating a GBM model
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Fitting the model
gbm.fit(X_train, y_train)

# Making predictions
y_pred_gbm = gbm.predict(X_test)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred_gbm)
print("Mean Squared Error:", mse)

plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred_gbm, label='GBM Net Predictions', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4, label='True Rate')
plt.title('Gradient Boosted Machine')
plt.xlabel('Actual Diabetes Rate')
plt.ylabel('Predicted Diabetes Rate')
plt.legend()
plt.show()

In [ ]:
#Make a feature importance plot
feature_importances = gbm.feature_importances_
features = X.columns
indices = np.argsort(feature_importances)

# Make a fancier plot
plt.figure(figsize=(10, 5))
plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], align='center', edgecolor='black')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

FORECASTING USING ARIMA MODEL

In [ ]:
# necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

**Plan:**

Data Preparation: Use the provided data sections to gather relevant data.

Stationarity Check: Ensure the data is stationary.

ARIMA Model: Build and fit ARIMA models to forecast economic and health costs.

VAR Model: Build and fit VAR models to capture the relationships between multiple time series.

**Data Prep:**

Include all these data components (will add on maybe)

1. Fast Food Stock Prices

2. Obesity Rates

3. Unemployment Rates

4. Meat Production (Red Meat, Poultry)

Maybe...

5. Commodity Prices (Corn, Coffee, Sugar)

6. Physical Activity Data

In [ ]:
# all datasets in one place
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")
meat_stats_slaughter_weights = import_dataset("1IHd-9p3aDoxbER3oBCZqBY23VVD5LOnu", "meat_stats_slaughter_weights")
meat_stats_slaughter_counts = import_dataset("1Eo58CgEeIk7Hnw_7I2yY46Ng6qiBLyeX", "meat_stats_slaughter_counts")
meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
all_commodities = import_dataset("1E8ELVRv2OFMbXwWYp2XVE1wH6Or6kPkB", "all_commodities")
acs5yr = import_dataset("1JFpNj9SlUiWwZTFc-pnlggKu6T7h_urP", "acs5yr")
agri_variable_list = import_dataset("1Y6Dv0yA8UXN5V5NhSdwUpT7JtzKj3oWm", "agri_variable_list")
agri_supplemental_county = import_dataset("1eM9e49xO841V-iR7wvFl1DPE-I8IRAKl", "agri_supplemental_county")
agri_supplemental_state = import_dataset("1BJsIiC3yPwtps1dA_-Ex-rUKzurce4c6", "agri_supplemental_state")
agri_state_and_county = import_dataset("1OFtRcynXKbMHFezP4ZLE5DEp6QUIGnHF", "agri_state_and_county")
employment = import_dataset("1x2uh0JILYFsarSP1h6Dtars79CtVYIP2", "employment")
health_region = import_dataset("18Y813FCdS2dXJSxOxUwcyVaAvwIxbOx-", "health_region")
social_determinants_of_health = import_dataset("17enYtkP62akioKX4_W9aup3pIiOyuwM8", "social_determinants_of_health")
# utf-8 encoding
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

try:
    us_census_2020_2023 = import_dataset("1JHgbYOHYHR-Bdd1-NAv6laH8xMFZQJTq", "us_census_2020_2023", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2020_2023 with 'latin1' encoding")

try:
    us_census_2010_2020 = import_dataset("1aFonlFnV2dQKS4VgO4kgmsUaxYctWp8e", "us_census_2010_2020", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2010_2020 with 'latin1' encoding")

try:
    us_census_2000_2010 = import_dataset("1pWz7cBJmTBitrY6GJ_mwyISFijdbDIbK", "us_census_2000_2010", encoding='latin1')
except UnicodeDecodeError:
    print("Failed to decode us_census_2000_2010 with 'latin1' encoding")
clean = import_dataset('1Pi2g3ep0xoGuxJbqt6iHqYWoc3PRgOOq', 'clean')

KeyboardInterrupt: 

In [ ]:
# print info about all the data
def print_dataset_info(dataset, dataset_name):
    print(f"Dataset: {dataset_name}")
    print(dataset.info())
    print("\nSample Data:")
    print(dataset.head())
    print("\n")

print_dataset_info(stock_descriptions, "stock_descriptions")
print_dataset_info(nutrition, "nutrition")
print_dataset_info(meat_stats_slaughter_weights, "meat_stats_slaughter_weights")
print_dataset_info(meat_stats_slaughter_counts, "meat_stats_slaughter_counts")
print_dataset_info(meat_stats_meat_production, "meat_stats_meat_production")
print_dataset_info(meat_stats_cold_storage, "meat_stats_cold_storage")
print_dataset_info(all_stocks, "all_stocks")
print_dataset_info(all_commodities, "all_commodities")
print_dataset_info(acs5yr, "acs5yr")
print_dataset_info(agri_variable_list, "agri_variable_list")
print_dataset_info(agri_supplemental_county, "agri_supplemental_county")
print_dataset_info(agri_supplemental_state, "agri_supplemental_state")
print_dataset_info(agri_state_and_county, "agri_state_and_county")
print_dataset_info(employment, "employment")
print_dataset_info(health_region, "health_region")
print_dataset_info(social_determinants_of_health, "social_determinants_of_health")
print_dataset_info(us_census_2020_2023, "us_census_2020_2023")
print_dataset_info(us_census_2010_2020, "us_census_2010_2020")
print_dataset_info(us_census_2000_2010, "us_census_2000_2010")
print_dataset_info(clean, "clean")

In [ ]:
# unemployment
url = "https://drive.google.com/uc?id=1x2uh0JILYFsarSP1h6Dtars79CtVYIP2"
employment = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

employment.index = pd.to_datetime(employment.index)
employment = employment.asfreq('MS')  # Set frequency to start of month

employment_rate = employment['Employment Rate']

# plot time series
plt.figure(figsize=(10, 6))
plt.plot(employment_rate, label='Employment Rate')
plt.title('Employment Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Employment Rate')
plt.legend()
plt.show()

# check stationarity using Augmented Dickey-Fuller test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    for key, value in result[4].items():
        print('Critical Value {}: {:.3f}'.format(key, value))

adf_test(employment_rate)

# failed the ADF test so differentiate
# first-order
employment_rate_diff = employment_rate.diff().dropna()

print("First-order differencing:")
adf_test(employment_rate_diff)

# second-order
employment_rate_diff2 = employment_rate.diff().diff().dropna()

print("\nSecond-order differencing:")
adf_test(employment_rate_diff2) # now it's stationary (p-value less than 0.05)

# plot second-order diff
employment_rate_diff2.plot(title='Second-order Differenced Employment Rate Over Time')
plt.show()

# figure out values for p and q
# plot ACF and PACF for the second-order differenced series
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_acf(employment_rate_diff2, ax=axes[0])
axes[0].set_title('ACF Plot')
plot_pacf(employment_rate_diff2, ax=axes[1])
axes[1].set_title('PACF Plot')

plt.show() # shows spikes - usually the higher the spike the greater the lag contributes

# ARIMA model for unemployment
# fit ARIMA model (p=2, d=2, q=1)

model = ARIMA(employment_rate, order=(2, 2, 1))
model_fit = model.fit()

print(model_fit.summary())

forecast_steps = 12  # Number of months to forecast
forecast = model_fit.forecast(steps=forecast_steps)

# plot forecase
plt.figure(figsize=(10, 6))
plt.plot(employment_rate, label='Observed')
plt.plot(forecast, label='Forecast', color='red')
plt.title('Employment Rate Forecast')
plt.legend()
plt.show()

In [ ]:
# fast food stocks
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')

fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

fast_food_stock_prices['Date'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices.set_index('Date', inplace=True)

# monthly
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()

plt.figure(figsize=(10, 6))
plt.plot(monthly_fast_food_stock_prices, label='Monthly Fast Food Stock Close Price')
plt.title('Monthly Fast Food Stock Close Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

# Check stationarity using Augmented Dickey-Fuller test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    for key, value in result[4].items():
        print('Critical Value {}: {:.3f}'.format(key, value))

adf_test(monthly_fast_food_stock_prices)

# Check stationarity using Augmented Dickey-Fuller test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    for key, value in result[4].items():
        print('Critical Value {}: {:.3f}'.format(key, value))

adf_test(monthly_fast_food_stock_prices)

# first-order
monthly_fast_food_stock_prices_diff = monthly_fast_food_stock_prices.diff().dropna()

print("First-order differencing:")
adf_test(monthly_fast_food_stock_prices_diff)

# second order
monthly_fast_food_stock_prices_diff2 = monthly_fast_food_stock_prices_diff.diff().dropna()

print("\nSecond-order differencing:")
adf_test(monthly_fast_food_stock_prices_diff2)  # now it's stationary (p-value less than 0.05)

# plot second-order differenced series
plt.figure(figsize=(10, 6))
plt.plot(monthly_fast_food_stock_prices_diff2, label='Second-order Differenced Close Price')
plt.title('Second-order Differenced Close Price Over Time')
plt.legend()
plt.show()

# Determine p and q values using ACF and PACF plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_acf(monthly_fast_food_stock_prices_diff2, ax=axes[0])
axes[0].set_title('ACF Plot')
plot_pacf(monthly_fast_food_stock_prices_diff2, ax=axes[1])
axes[1].set_title('PACF Plot')

plt.show()

# fit ARIMA
model = ARIMA(monthly_fast_food_stock_prices, order=(2, 2, 1))
model_fit = model.fit()

print(model_fit.summary())

forecast_steps = 12  # Number of months to forecast
forecast = model_fit.forecast(steps=forecast_steps)

# Plot forecast
plt.figure(figsize=(10, 6))
plt.plot(monthly_fast_food_stock_prices, label='Observed')
plt.plot(forecast, label='Forecast', color='red')
plt.title('Fast Food Stock Close Price Forecast')
plt.legend()
plt.show()


In [ ]:
meat_stats_cold_storage = pd.read_csv('meat_stats_cold_storage.csv')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])

# Aggregate data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([meat_stats_cold_storage['Date'].dt.to_period('M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']
cold_storage_agg.index = cold_storage_agg.index.to_timestamp()

# Plot time series for red meat and poultry
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_agg['ColdStorageRedMeat'], label='Cold Storage Red Meat')
plt.plot(cold_storage_agg['ColdStoragePoultry'], label='Cold Storage Poultry')
plt.title('Cold Storage Over Time')
plt.xlabel('Date')
plt.ylabel('Weight (Million Pounds)')
plt.legend()
plt.show()

# Function to perform ADF test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    for key, value in result[4].items():
        print('Critical Value {}: {:.3f}'.format(key, value))

# Red Meat Analysis
print("\nCold Storage Red Meat Analysis:")
adf_test(cold_storage_agg['ColdStorageRedMeat'])

# First-order differencing
cold_storage_red_meat_diff = cold_storage_agg['ColdStorageRedMeat'].diff().dropna()

print("First-order differencing:")
adf_test(cold_storage_red_meat_diff)

# Plot first-order differenced series
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_red_meat_diff, label='First-order Differenced Cold Storage Red Meat')
plt.title('First-order Differenced Cold Storage Red Meat Over Time')
plt.legend()
plt.show()

# Determine p and q values using ACF and PACF plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_acf(cold_storage_red_meat_diff, ax=axes[0])
axes[0].set_title('ACF Plot - Red Meat')
plot_pacf(cold_storage_red_meat_diff, ax=axes[1])
axes[1].set_title('PACF Plot - Red Meat')

plt.show()

# Fit ARIMA model for Red Meat (p=2, d=2, q=1)
model_red_meat = ARIMA(cold_storage_agg['ColdStorageRedMeat'], order=(2, 1, 2))
model_fit_red_meat = model_red_meat.fit()

print(model_fit_red_meat.summary())

forecast_steps = 24  # months
forecast_red_meat = model_fit_red_meat.forecast(steps=forecast_steps)

# plot forecast for Red Meat
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_agg['ColdStorageRedMeat'], label='Observed - Red Meat')
plt.plot(forecast_red_meat, label='Forecast - Red Meat', color='red')
plt.title('Cold Storage Red Meat Forecast')
plt.legend()
plt.show()

# Poultry Analysis
print("\nCold Storage Poultry Analysis:")
adf_test(cold_storage_agg['ColdStoragePoultry'])

# First-order differencing
cold_storage_poultry_diff = cold_storage_agg['ColdStoragePoultry'].diff().dropna()

print("First-order differencing:")
adf_test(cold_storage_poultry_diff)

# Plot first-order differenced series
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_poultry_diff, label='First-order Differenced Cold Storage Poultry')
plt.title('First-order Differenced Cold Storage Poultry Over Time')
plt.legend()
plt.show()

# Determine p and q values using ACF and PACF plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plot_acf(cold_storage_poultry_diff, ax=axes[0])
axes[0].set_title('ACF Plot - Poultry')
plot_pacf(cold_storage_poultry_diff, ax=axes[1])
axes[1].set_title('PACF Plot - Poultry')

plt.show()

# Fit ARIMA model for Poultry (p=2, d=2, q=1)
model_poultry = ARIMA(cold_storage_agg['ColdStoragePoultry'], order=(2, 1, 2))
model_fit_poultry = model_poultry.fit()

print(model_fit_poultry.summary())

forecast_poultry_steps = 24  # Number of months to forecast
forecast_poultry = model_fit_poultry.forecast(steps=forecast_poultry_steps)

# Plot forecast for Poultry
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_agg['ColdStoragePoultry'], label='Observed - Poultry')
plt.plot(forecast_poultry, label='Forecast - Poultry', color='red')
plt.title('Cold Storage Poultry Forecast')
plt.legend()
plt.show()


In [ ]:
# ARIMA seasonality for cold storage meats
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

meat_stats_cold_storage = pd.read_csv('meat_stats_cold_storage.csv')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])
meat_stats_cold_storage.set_index('Date', inplace=True)

# aggregate data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# decompose for seasonal
decompose_red_meat = seasonal_decompose(cold_storage_agg['ColdStorageRedMeat'], model='additive', period=12)
decompose_red_meat.plot()
plt.show()

decompose_poultry = seasonal_decompose(cold_storage_agg['ColdStoragePoultry'], model='additive', period=12)
decompose_poultry.plot()
plt.show()

# seasonal differencing (to make the series stationary)
cold_storage_agg['RedMeat_diff'] = cold_storage_agg['ColdStorageRedMeat'].diff(12).dropna()
cold_storage_agg['Poultry_diff'] = cold_storage_agg['ColdStoragePoultry'].diff(12).dropna()

# check stationarity
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    for key, value in result[4].items():
        print('Critical Value {}: {:.3f}'.format(key, value))

print("ADF Test for Red Meat (Seasonal Differencing):")
adf_test(cold_storage_agg['RedMeat_diff'].dropna())

print("\nADF Test for Poultry (Seasonal Differencing):")
adf_test(cold_storage_agg['Poultry_diff'].dropna())

# ACF and PACF plots for seasonally differenced series
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
plot_acf(cold_storage_agg['RedMeat_diff'].dropna(), ax=axes[0])
axes[0].set_title('ACF Plot - Red Meat (Seasonal Differencing)')
plot_pacf(cold_storage_agg['RedMeat_diff'].dropna(), ax=axes[1])
axes[1].set_title('PACF Plot - Red Meat (Seasonal Differencing)')
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
plot_acf(cold_storage_agg['Poultry_diff'].dropna(), ax=axes[0])
axes[0].set_title('ACF Plot - Poultry (Seasonal Differencing)')
plot_pacf(cold_storage_agg['Poultry_diff'].dropna(), ax=axes[1])
axes[1].set_title('PACF Plot - Poultry (Seasonal Differencing)')
plt.show()

# fir SARIMA model for Red Meat
model_red_meat = SARIMAX(cold_storage_agg['ColdStorageRedMeat'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model_fit_red_meat = model_red_meat.fit(disp=False)
print(model_fit_red_meat.summary())

# forecast
forecast_steps = 24
forecast_red_meat = model_fit_red_meat.forecast(steps=forecast_steps)

# plot
plt.figure(figsize=(10, 6))
plt.plot(cold_storage_agg['ColdStorageRedMeat'], label='Observed - Red Meat')
plt.plot(forecast_red_meat, label='Forecast - Red Meat', color='red')
plt.title('Cold Storage Red Meat Forecast')
plt.legend()
plt.show()

# fit SARIMA model for Poultry
model_poultry = SARIMAX(cold_storage_agg['ColdStoragePoultry'], order=(1, 1, 1), seasonal_order=(2, 1, 1, 12))
model_fit_poultry = model_poultry.fit(disp=False)
print(model_fit_poultry.summary())

forecast_poultry = model_fit_poultry.forecast(steps=forecast_steps)

plt.figure(figsize=(10, 6))
plt.plot(cold_storage_agg['ColdStoragePoultry'], label='Observed - Poultry')
plt.plot(forecast_poultry, label='Forecast - Poultry', color='red')
plt.title('Cold Storage Poultry Forecast')
plt.legend()
plt.show()


VAR MODELS

In [ ]:
import pandas as pd

# Load data
meat_stats_cold_storage = pd.read_csv('meat_stats_cold_storage.csv')

# Print the first few rows of the DataFrame
print(meat_stats_cold_storage.head())

# Print the data types of each column
print(meat_stats_cold_storage.dtypes)

# Convert 'Date' column to datetime format and set as index
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'], errors='coerce')
print(meat_stats_cold_storage.head())
print(meat_stats_cold_storage.dtypes)

# Ensure only numeric columns are included
print(meat_stats_cold_storage['Weight'].unique())
print(meat_stats_cold_storage['Weight'].head())

numeric_meat_storage = meat_stats_cold_storage[['Weight']]
print(numeric_meat_storage.head())


In [ ]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt

meat_stats_meat_production = pd.read_csv('meat_stats_meat_production.csv')
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')

meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_meat_production.set_index('Date', inplace=True)

# convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

print("Missing values in meat production data before handling:")
print(meat_stats_meat_production.isnull().sum())
meat_stats_meat_production = meat_stats_meat_production.dropna()
print("Missing values in meat production data after handling:")
print(meat_stats_meat_production.isnull().sum())

print("Unique values in 'Animal' column:")
print(meat_stats_meat_production['Animal'].unique())
print("Unique values in 'Type of Meat' column:")
print(meat_stats_meat_production['Type of Meat'].unique())

# filter data for 'Red Meat'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']

print("Filtered red meat production data:")
print(red_meat_production.head())
print("Filtered red meat production data info:")
print(red_meat_production.info())

fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Date'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices.set_index('Date', inplace=True)

monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()

print("Resampled fast food stock prices monthly data:")
print(monthly_fast_food_stock_prices.head())

red_meat_production_monthly = red_meat_production.resample('M').mean()

print("Resampled red meat production monthly data:")
print(red_meat_production_monthly.head())

# common date ranges
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max())

print(f"Start date: {start_date}")
print(f"End date: {end_date}")

aligned_red_meat_production = red_meat_production_monthly.loc[start_date:end_date]
aligned_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]

print("Aligned red meat production monthly data:")
print(aligned_red_meat_production.head())
print("Aligned fast food stock prices monthly data:")
print(aligned_fast_food_stock_prices.head())

# combine into a single DataFrame
data = pd.concat([aligned_red_meat_production, aligned_fast_food_stock_prices], axis=1)
data.columns = ['RedMeatProduction', 'Fast_Food_Stock_Price']

data = data.dropna()

print("Missing values in combined data after resampling and dropping:")
print(data.isnull().sum())

print("Head of combined data:")
print(data.head())

# determin num of observations and adjust maxlags accordingly
num_observations = len(data)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # fit VAR model with the adjusted maxlags value
    model = VAR(data)
    results = model.fit(maxlags=maxlags, ic='aic')

    print(results.summary())

    # forecast
    forecast_steps = 12
    forecast = results.forecast(data.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=['RedMeatProduction', 'Fast_Food_Stock_Price'])

    # plot
    plt.figure(figsize=(10, 6))
    plt.plot(data['RedMeatProduction'], label='Observed Red Meat Production')
    plt.plot(forecast_df['RedMeatProduction'], label='Forecast Red Meat Production', linestyle='--')
    plt.plot(data['Fast_Food_Stock_Price'], label='Observed Fast Food Stock Price')
    plt.plot(forecast_df['Fast_Food_Stock_Price'], label='Forecast Fast Food Stock Price', linestyle='--')
    plt.title('Red Meat Production and Fast Food Stock Price Forecast')
    plt.xlabel('Date')
    plt.ylabel('Values')
    plt.legend()
    plt.show()

    # Granger Causality Test
    max_lag = 2
    test_result = grangercausalitytests(data[['Fast_Food_Stock_Price', 'RedMeatProduction']], maxlag=max_lag, verbose=True)

    # extract p-values from Granger Causality Test results
    granger_p_values = {key: val[0]['ssr_ftest'][1] for key, val in test_result.items()}
    print("Granger Causality Test p-values:", granger_p_values)
else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt

# Load data
meat_stats_meat_production = pd.read_csv('meat_stats_meat_production.csv')
all_stocks = pd.read_csv('all_stocks.csv')
stock_descriptions = pd.read_csv('stock_descriptions.csv')

# Check initial data
print("Initial meat_stats_meat_production:")
print(meat_stats_meat_production.head())
print("Data types of columns in meat_stats_meat_production:")
print(meat_stats_meat_production.dtypes)

# Investigate 'Production' column
print("Sample values from 'Production' column:")
print(meat_stats_meat_production['Production'].head(20))

# Preprocess meat production data
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_meat_production.set_index('Date', inplace=True)

# Check for missing values in meat production data
print("Missing values in meat production data before handling:")
print(meat_stats_meat_production.isnull().sum())

# Handle missing values by dropping them
meat_stats_meat_production = meat_stats_meat_production.dropna()

# Verify no missing values remain
print("Missing values in meat production data after handling:")
print(meat_stats_meat_production.isnull().sum())

# Print unique values in 'Animal' column to verify filtering
print("Unique values in 'Animal' column:")
print(meat_stats_meat_production['Animal'].unique())

# Filter data for 'Red Meat'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Animal'] == 'Red Meat', 'Production'].astype(float)

# Print filtered red meat production data
print("Filtered red meat production data:")
print(red_meat_production.head())

# Preprocess stock data
print("Initial all_stocks data:")
print(all_stocks.head())

fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]
fast_food_stock_prices['Date'] = pd.to_datetime(fast_food_stock_prices['Date-Time'])
fast_food_stock_prices.set_index('Date', inplace=True)

# Print preprocessed fast food stock prices data
print("Preprocessed fast food stock prices data:")
print(fast_food_stock_prices.head())

# Resample to monthly data and take mean of 'Close' price
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()

# Resample red meat production to monthly data
red_meat_production_monthly = red_meat_production.resample('M').mean()

# Print resampled data
print("Resampled red meat production monthly data:")
print(red_meat_production_monthly.head())
print("Resampled fast food stock prices monthly data:")
print(monthly_fast_food_stock_prices.head())

# Align both time series to the same time range
start_date = max(red_meat_production_monthly.index.min(), monthly_fast_food_stock_prices.index.min())
end_date = min(red_meat_production_monthly.index.max(), monthly_fast_food_stock_prices.index.max())

print(f"Start date: {start_date}")
print(f"End date: {end_date}")

red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]

# Print aligned data
print("Aligned red meat production monthly data:")
print(red_meat_production_monthly.head())
print("Aligned fast food stock prices monthly data:")
print(monthly_fast_food_stock_prices.head())

# Combine into a single DataFrame
data = pd.concat([red_meat_production_monthly, monthly_fast_food_stock_prices], axis=1)
data.columns = ['RedMeatProduction', 'Fast_Food_Stock_Price']

# Drop any remaining missing values
data = data.dropna()

# Ensure no missing values
print("Missing values in combined data after resampling and dropping:")
print(data.isnull().sum())

# Print combined data
print("Head of combined data:")
print(data.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(data)
maxlags = num_observations // 3  # A rule of thumb is to use a maxlag that is a fraction of the total number of observations

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

# Check if we have enough observations to fit the model
if num_observations > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Forecast the next 12 months
    forecast_steps = 12
    forecast = results.forecast(data.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=['RedMeatProduction', 'Fast_Food_Stock_Price'])

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(data['RedMeatProduction'], label='Observed Red Meat Production')
    plt.plot(forecast_df['RedMeatProduction'], label='Forecast Red Meat Production', linestyle='--')
    plt.plot(data['Fast_Food_Stock_Price'], label='Observed Fast Food Stock Price')
    plt.plot(forecast_df['Fast_Food_Stock_Price'], label='Forecast Fast Food Stock Price', linestyle='--')
    plt.title('Red Meat Production and Fast Food Stock Price Forecast')
    plt.xlabel('Date')
    plt.ylabel('Values')
    plt.legend()
    plt.show()

    # Granger Causality Test
    max_lag = 2
    test_result = grangercausalitytests(data[['Fast_Food_Stock_Price', 'RedMeatProduction']], maxlag=max_lag, verbose=True)

    # Extract p-values from Granger Causality Test results
    granger_p_values = {key: val[0]['ssr_ftest'][1] for key, val in test_result.items()}
    print("Granger Causality Test p-values:", granger_p_values)
else:
    print("Not enough observations to fit the model.")


In [ ]:
# VAR of Fast Food Stock Prices, Poultry Production, Red Meat Production, Cold Storage Poultry, Cold Storage Red Meat Levels
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")
employment = import_dataset("1x2uh0JILYFsarSP1h6Dtars79CtVYIP2", "employment")

# Convert 'Date' columns to datetime format
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])
employment['Date'] = pd.to_datetime(employment['Date'])

# Set 'Date' as index
meat_stats_meat_production.set_index('Date', inplace=True)
meat_stats_cold_storage.set_index('Date', inplace=True)
all_stocks.set_index('Date', inplace=True)
employment.set_index('Date', inplace=True)

# Convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

# Handle missing values
meat_stats_meat_production = meat_stats_meat_production.dropna()
meat_stats_cold_storage = meat_stats_cold_storage.dropna()
employment = employment.dropna()

# Filter data for 'Red Meat' and 'Poultry'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']
poultry_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Poultry', 'Production']

# Filter fast food stocks
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Resample to monthly data
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()
red_meat_production_monthly = red_meat_production.resample('M').mean()
poultry_production_monthly = poultry_production.resample('M').mean()

# Aggregate cold storage data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# Align both time series to the same time range
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min(), poultry_production_monthly.index.min(), cold_storage_agg.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max(), poultry_production_monthly.index.max(), cold_storage_agg.index.max())

monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]
red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
poultry_production_monthly = poultry_production_monthly.loc[start_date:end_date]
cold_storage_agg = cold_storage_agg.loc[start_date:end_date]

# Combine into a single DataFrame
data = pd.concat([monthly_fast_food_stock_prices, red_meat_production_monthly, poultry_production_monthly, cold_storage_agg], axis=1)
data.columns = ['Fast_Food_Stock_Price', 'RedMeatProduction', 'PoultryProduction', 'ColdStorageRedMeat', 'ColdStoragePoultry']

# Drop any remaining missing values
data = data.dropna()

# Add employment data
employment_rate = employment['Unemployment Rate'].resample('M').mean().loc[start_date:end_date]
data['Unemployment_Rate'] = employment_rate

# Ensure no NaN or infinite values
data = data.replace([np.inf, -np.inf], np.nan).dropna()

# Check if there are any NaNs or infinite values left
print("Missing values in combined data after resampling and dropping:")
print(data.isnull().sum())

print("Head of combined data:")
print(data.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(data)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Forecast the next 12
    forecast_steps = 12
    forecast = results.forecast(data.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=data.columns)

    # Extract predicted unemployment rates
    predicted_unemployment = forecast_df['Unemployment_Rate']

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(data['Unemployment_Rate'], label='Observed Unemployment Rate')
    plt.plot(forecast_index, predicted_unemployment, label='Forecasted Unemployment Rate', linestyle='--')
    plt.title('Forecasted Unemployment Rate')
    plt.xlabel('Date')
    plt.ylabel('Unemployment Rate')
    plt.legend()
    plt.show()

    # Granger Causality Test
    max_lag = 2
    test_result = grangercausalitytests(data[['Fast_Food_Stock_Price', 'Unemployment_Rate']], maxlag=max_lag, verbose=True)

    # Extract p-values from Granger Causality Test results
    granger_p_values = {key: val[0]['ssr_ftest'][1] for key, val in test_result.items()}
    print("Granger Causality Test p-values:", granger_p_values)
else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

# Load the new unemployment data
unemployment = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Display the first few rows of the dataset
print(unemployment.head())

# Convert 'DATE' column to datetime format
unemployment['DATE'] = pd.to_datetime(unemployment['DATE'])

# Set 'DATE' as index
unemployment.set_index('DATE', inplace=True)

# Filter the data between 2007 and 2020
unemployment_filtered = unemployment.loc['2007-01-01':'2020-12-31']

# Plot the unemployment data
plt.figure(figsize=(10, 6))
plt.plot(unemployment_filtered.index, unemployment_filtered['LNS14000036'], label='Unemployment Rate')
plt.title('Unemployment Rate (2007-2020)')
plt.xlabel('Date')
plt.ylabel('Unemployment Rate')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

# unemployment data
unemployment = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

print(unemployment.head())

# convert 'DATE' column to datetime format
unemployment['DATE'] = pd.to_datetime(unemployment['DATE'])

unemployment.set_index('DATE', inplace=True)

# plot
plt.figure(figsize=(10, 6))
plt.plot(unemployment.index, unemployment['LNS14000036'], label='Unemployment Rate')
plt.title('Unemployment Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Unemployment Rate')
plt.legend()
plt.show()

In [ ]:
# var analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")

# Convert 'Date' columns to datetime format
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'])
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])

# Set 'Date' as index
meat_stats_meat_production.set_index('Date', inplace=True)
meat_stats_cold_storage.set_index('Date', inplace=True)
all_stocks.set_index('Date', inplace=True)

# Convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

# Handle missing values
meat_stats_meat_production = meat_stats_meat_production.dropna()
meat_stats_cold_storage = meat_stats_cold_storage.dropna()

# Filter data for 'Red Meat' and 'Poultry'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']
poultry_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Poultry', 'Production']

# Filter fast food stocks
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Resample to monthly data
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()
red_meat_production_monthly = red_meat_production.resample('M').mean()
poultry_production_monthly = poultry_production.resample('M').mean()

# Aggregate cold storage data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# Align both time series to the same time range
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min(), poultry_production_monthly.index.min(), cold_storage_agg.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max(), poultry_production_monthly.index.max(), cold_storage_agg.index.max())

monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]
red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
poultry_production_monthly = poultry_production_monthly.loc[start_date:end_date]
cold_storage_agg = cold_storage_agg.loc[start_date:end_date]

# Combine into a single DataFrame
data = pd.concat([monthly_fast_food_stock_prices, red_meat_production_monthly, poultry_production_monthly, cold_storage_agg], axis=1)
data.columns = ['Fast_Food_Stock_Price', 'RedMeatProduction', 'PoultryProduction', 'ColdStorageRedMeat', 'ColdStoragePoultry']

# Drop any remaining missing values
data = data.dropna()

# Add the new unemployment data
unemployment_rate = unemployment['LNS14000036'].resample('M').mean()
data['Unemployment_Rate'] = unemployment_rate

# Ensure no NaN or infinite values
data = data.replace([np.inf, -np.inf], np.nan).dropna()

# Check if there are any NaNs or infinite values left
print("Missing values in combined data after resampling and dropping:")
print(data.isnull().sum())

print("Head of combined data:")
print(data.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(data)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Forecast the next 12
    forecast_steps = 12
    forecast = results.forecast(data.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=data.columns)

    # Extract predicted unemployment rates
    predicted_unemployment = forecast_df['Unemployment_Rate']

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(data['Unemployment_Rate'], label='Observed Unemployment Rate')
    plt.plot(forecast_index, predicted_unemployment, label='Forecasted Unemployment Rate', linestyle='--')
    plt.title('Forecasted Unemployment Rate')
    plt.xlabel('Date')
    plt.ylabel('Unemployment Rate')
    plt.legend()
    plt.show()

    # Granger Causality Test with specific lags based on the table
    causality_tests = {
        'Fast_Food_Stock_Price -> Unemployment_Rate': 3,
        'Fast_Food_Stock_Price -> PoultryProduction': 1,
        'ColdStoragePoultry -> RedMeatProduction': 5,
        'ColdStoragePoultry -> Unemployment_Rate': 1,
        'ColdStoragePoultry -> Fast_Food_Stock_Price': 1,
        'RedMeatProduction -> Fast_Food_Stock_Price': 2,
        'RedMeatProduction -> ColdStorageRedMeat': 5
    }

    for test, lags in causality_tests.items():
        x, y = test.split(' -> ')
        print(f'\nGranger Causality Test: {x} causes {y} with {lags} lags')
        test_result = grangercausalitytests(data[[x, y]], maxlag=lags, verbose=False)
        p_values = {key: val[0]['ssr_ftest'][1] for key, val in test_result.items()}
        print(f'p-values for lags: {p_values}')
else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding)

meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'Date' columns to datetime format
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'], format='%b-%Y')
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'Date' as index
meat_stats_meat_production.set_index('Date', inplace=True)
meat_stats_cold_storage.set_index('Date', inplace=True)
all_stocks.set_index('Date', inplace=True)
unemployment_new.set_index('DATE', inplace=True)

# Convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

# Handle missing values
meat_stats_meat_production = meat_stats_meat_production.dropna()
meat_stats_cold_storage = meat_stats_cold_storage.dropna()

# Filter data for 'Red Meat' and 'Poultry'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']
poultry_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Poultry', 'Production']

# Filter fast food stocks
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Resample to monthly data
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()
red_meat_production_monthly = red_meat_production.resample('M').mean()
poultry_production_monthly = poultry_production.resample('M').mean()

# Aggregate cold storage data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# Align both time series to the same time range
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min(), poultry_production_monthly.index.min(), cold_storage_agg.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max(), poultry_production_monthly.index.max(), cold_storage_agg.index.max())

monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]
red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
poultry_production_monthly = poultry_production_monthly.loc[start_date:end_date]
cold_storage_agg = cold_storage_agg.loc[start_date:end_date]

# Combine into a single DataFrame
data = pd.concat([monthly_fast_food_stock_prices, red_meat_production_monthly, poultry_production_monthly, cold_storage_agg], axis=1)
data.columns = ['Fast_Food_Stock_Price', 'RedMeatProduction', 'PoultryProduction', 'ColdStorageRedMeat', 'ColdStoragePoultry']

# Drop any remaining missing values
data = data.dropna()

# Add the new unemployment data
unemployment_rate = unemployment_new['LNS14000036'].resample('M').mean()
data['Unemployment_Rate'] = unemployment_rate

# Filter out the COVID-19 spike for the VAR analysis
covid_start_date = '2020-04-01'
covid_end_date = '2022-06-01'
data_for_var = pd.concat([data.loc[:covid_start_date], data.loc[covid_end_date:]])

# Ensure no NaN or infinite values
data_for_var = data_for_var.replace([np.inf, -np.inf], np.nan).dropna()

# Check if there are any NaNs or infinite values left
print("Missing values in combined data after resampling and dropping:")
print(data_for_var.isnull().sum())

print("Head of combined data:")
print(data_for_var.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(data_for_var)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(data_for_var)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Forecast the next 12
    forecast_steps = 12
    forecast = results.forecast(data_for_var.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=data_for_var.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=data.columns)

    # Extract predicted unemployment rates
    predicted_unemployment = forecast_df['Unemployment_Rate']

    # Plot the forecast
    plt.figure(figsize=(10, 6))
    plt.plot(data['Unemployment_Rate'], label='Observed Unemployment Rate')
    plt.plot(forecast_index, predicted_unemployment, label='Forecasted Unemployment Rate', linestyle='--')
    plt.title('Forecasted Unemployment Rate')
    plt.xlabel('Date')
    plt.ylabel('Unemployment Rate')
    plt.legend()
    plt.show()

    # Granger Causality Test with specific lags based on the table
    causality_tests = {
        'Fast_Food_Stock_Price -> Unemployment_Rate': 3,
        'Fast_Food_Stock_Price -> PoultryProduction': 1,
        'ColdStoragePoultry -> RedMeatProduction': 5,
        'ColdStoragePoultry -> Unemployment_Rate': 1,
        'ColdStoragePoultry -> Fast_Food_Stock_Price': 1,
        'RedMeatProduction -> Fast_Food_Stock_Price': 2,
        'RedMeatProduction -> ColdStorageRedMeat': 5
    }

    for test, lags in causality_tests.items():
        x, y = test.split(' -> ')
        print(f'\nGranger Causality Test: {x} causes {y} with {lags} lags')
        test_result = grangercausalitytests(data_for_var[[x, y]], maxlag=lags, verbose=True)
        p_values = {key: val[0]['ssr_ftest'][1] for key, val in test_result.items()}
        print(f'p-values for lags: {p_values}')
else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Plot the generated monthly obesity data
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Generated Monthly Obesity Rate')
plt.title('Generated Monthly Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

# Print the first few rows of the generated monthly obesity data
print("Generated Monthly Obesity Data:")
print(monthly_obesity_data.head())

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Print the first few rows of the monthly unemployment data
print("Monthly Unemployment Data:")
print(monthly_unemployment_rate.head())

# Combine obesity and unemployment data into a single DataFrame
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).dropna()

# Print the combined data before dropping NA
print("Combined Data Before Dropping NA:")
print(pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).head(30))

# Ensure no NaN or infinite values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Check for missing values
print("Missing values in combined data after resampling and dropping:")
print(combined_data.isnull().sum())

# Print the head of the combined data
print("Head of combined data:")
print(combined_data.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(combined_data)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(combined_data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Split the data into train and test sets
    n_train = int(0.8 * len(combined_data))
    train_data = combined_data.iloc[:n_train]
    test_data = combined_data.iloc[n_train:]

    # Fit VAR model on the training data
    model = VAR(train_data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Forecast the test period
    forecast_steps = len(test_data)
    forecast = results.forecast(train_data.values[-results.k_ar:], steps=forecast_steps)
    forecast_index = pd.date_range(start=train_data.index[-1] + pd.DateOffset(months=1), periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=combined_data.columns)

    # Extract predicted obesity rates
    predicted_obesity = forecast_df['Obesity_Rate']

    # Plot the observed and forecasted obesity rates
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Obesity_Rate'], label='Observed Obesity Rate')
    plt.plot(forecast_index, predicted_obesity, label='Forecasted Obesity Rate', linestyle='--')
    plt.title('Observed and Forecasted Obesity Rate')
    plt.xlabel('Date')
    plt.ylabel('Obesity Rate')
    plt.legend()
    plt.show()

    # Calculate and print the Mean Squared Error
    mse = mean_squared_error(test_data['Obesity_Rate'], predicted_obesity)
    print(f"Mean Squared Error: {mse}")

else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import mean_squared_error

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Plot the generated monthly obesity data
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Generated Monthly Obesity Rate')
plt.title('Generated Monthly Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

# Print the first few rows of the generated monthly obesity data
print("Generated Monthly Obesity Data:")
print(monthly_obesity_data.head())

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Print the first few rows of the monthly unemployment data
print("Monthly Unemployment Data:")
print(monthly_unemployment_rate.head())

# Combine obesity and unemployment data into a single DataFrame
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).dropna()

# Print the combined data before dropping NA
print("Combined Data Before Dropping NA:")
print(pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).head(30))

# Ensure no NaN or infinite values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Check for missing values
print("Missing values in combined data after resampling and dropping:")
print(combined_data.isnull().sum())

# Print the head of the combined data
print("Head of combined data:")
print(combined_data.head())

# Determine the number of observations and adjust maxlags accordingly
num_observations = len(combined_data)
maxlags = min(15, num_observations // 3)  # Use a smaller maxlag

print(f"Number of observations: {num_observations}")
print(f"Maxlags set to: {maxlags}")

if num_observations > 0 and maxlags > 0:
    # Fit VAR model with the adjusted maxlags value
    model = VAR(combined_data)
    results = model.fit(maxlags=maxlags, ic='aic')

    # Print summary of the model
    print(results.summary())

    # Forecast the entire historical period
    forecast_steps = len(combined_data)
    forecast = results.forecast(combined_data.values[:results.k_ar], steps=forecast_steps)
    forecast_index = pd.date_range(start=combined_data.index[results.k_ar], periods=forecast_steps, freq='M')
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=combined_data.columns)

    # Extract predicted obesity rates
    predicted_obesity = forecast_df['Obesity_Rate']

    # Plot the observed and forecasted obesity rates
    plt.figure(figsize=(10, 6))
    plt.plot(combined_data['Obesity_Rate'], label='Observed Obesity Rate')
    plt.plot(forecast_index, predicted_obesity, label='Forecasted Obesity Rate', linestyle='--')
    plt.title('Observed and Forecasted Obesity Rate')
    plt.xlabel('Date')
    plt.ylabel('Obesity Rate')
    plt.legend()
    plt.show()

    # Calculate and print the Mean Squared Error
    mse = mean_squared_error(combined_data['Obesity_Rate'][results.k_ar:], predicted_obesity)
    print(f"Mean Squared Error: {mse}")

else:
    print("Not enough observations to fit the model.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Combine obesity and unemployment data into a single DataFrame
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).dropna()

# Ensure no NaN or infinite values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

# Prepare the data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length, :-1]
        y = data[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 12
X, y = create_sequences(scaled_data, seq_length)

# Split the data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_test), scaled_data.shape[1]-1)), y_test.reshape(-1, 1)), axis=1))[:, -1]
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_pred), scaled_data.shape[1]-1)), y_pred), axis=1))[:, -1]

# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index[-len(y_test):], y_test_inv, label='Observed Obesity Rate')
plt.plot(monthly_obesity_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

# Calculate and print the Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Apply a rolling mean to smooth the data
obesity_data['Obesity_Rate'] = obesity_data['Obesity_Rate'].rolling(window=3, center=True).mean()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Add some noise to the monthly obesity rate
np.random.seed(42)
monthly_obesity_rate += np.random.normal(scale=0.5, size=monthly_obesity_rate.shape)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Load additional datasets
meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")

# Convert 'Date' columns to datetime format
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'], format='%b-%Y')
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])

# Set 'Date' as index
meat_stats_meat_production.set_index('Date', inplace=True)
meat_stats_cold_storage.set_index('Date', inplace=True)
all_stocks.set_index('Date', inplace=True)

# Convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

# Handle missing values
meat_stats_meat_production = meat_stats_meat_production.dropna()
meat_stats_cold_storage = meat_stats_cold_storage.dropna()

# Filter data for 'Red Meat' and 'Poultry'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']
poultry_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Poultry', 'Production']

# Filter fast food stocks
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Resample to monthly data
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()
red_meat_production_monthly = red_meat_production.resample('M').mean()
poultry_production_monthly = poultry_production.resample('M').mean()

# Aggregate cold storage data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# Align both time series to the same time range
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min(), poultry_production_monthly.index.min(), cold_storage_agg.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max(), poultry_production_monthly.index.max(), cold_storage_agg.index.max())

monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]
red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
poultry_production_monthly = poultry_production_monthly.loc[start_date:end_date]
cold_storage_agg = cold_storage_agg.loc[start_date:end_date]

# Combine all data
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate, monthly_fast_food_stock_prices,
                           red_meat_production_monthly, poultry_production_monthly, cold_storage_agg], axis=1).dropna()

# Print the combined data before further analysis
print("Combined Data Before Dropping NA:")
print(combined_data.head(30))

# Print missing values in the combined data
print("Missing values in combined data after resampling and dropping:")
print(combined_data.isnull().sum())

# Print the head of the combined data
print("Head of combined data:")
print(combined_data.head())



In [ ]:
print(monthly_obesity_data)
# Plot the obesity rate
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Obesity Rate')
plt.title('Monthly Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout

# Prepare the data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length, :]
        y = data[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 24  # Increased sequence length
X, y = create_sequences(scaled_data, seq_length)

# Split the data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and test data
y_test_full = np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1)
y_pred_full = np.concatenate((X_test[:, -1, :-1], y_pred), axis=1)

# Inverse transform to get the original scale
y_test_inv = scaler.inverse_transform(y_test_full)[:, -1]
y_pred_inv = scaler.inverse_transform(y_pred_full)[:, -1]

# Rescale the obesity rate to be in percentage format (if needed)
y_test_inv = y_test_inv / 10
y_pred_inv = y_pred_inv / 10

# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(combined_data.index[-len(y_test):], y_test_inv, label='Observed Obesity Rate')
plt.plot(combined_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate (%)')
plt.legend()
plt.show()

# Calculate and print the Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")


In [ ]:
# Plot the combined graph
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Observed Obesity Rate')
plt.plot(combined_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate (%)')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.layers import GRU

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(combined_data)

# Prepare the data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length, :]
        y = data[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 24  # Increased sequence length
X, y = create_sequences(scaled_data, seq_length)

# Split the data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(GRU(100, activation='relu', return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and test data
y_test_full = np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1)
y_pred_full = np.concatenate((X_test[:, -1, :-1], y_pred), axis=1)

# Inverse transform to get the original scale
y_test_inv = scaler.inverse_transform(y_test_full)[:, -1]
y_pred_inv = scaler.inverse_transform(y_pred_full)[:, -1]

# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Observed Obesity Rate')
plt.plot(combined_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate (%)')
plt.legend()
plt.show()

# Calculate and print the Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")


In [ ]:
# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(combined_data.index[-len(y_test):], y_test_inv / 10, label='Observed Obesity Rate')
plt.plot(combined_data.index[-len(y_test):], y_pred_inv / 10, label='Predicted Obesity Rate', linestyle='--')
plt.plot(monthly_obesity_data.index, monthly_obesity_data['Obesity_Rate'], label='Observed Obesity Rate')

plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate (%)')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Apply a rolling mean to smooth the data
obesity_data['Obesity_Rate'] = obesity_data['Obesity_Rate'].rolling(window=3, center=True).mean()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Add some noise to the monthly obesity rate
np.random.seed(42)
monthly_obesity_rate += np.random.normal(scale=0.5, size=monthly_obesity_rate.shape)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Align the date range of both datasets
aligned_dates = monthly_obesity_data.index.intersection(monthly_unemployment_rate.index)
monthly_obesity_data = monthly_obesity_data.loc[aligned_dates]
monthly_unemployment_rate = monthly_unemployment_rate.loc[aligned_dates]

# Combine obesity and unemployment data into a single DataFrame
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate], axis=1).dropna()

# Ensure no NaN or infinite values
combined_data = combined_data.replace([np.inf, -np.inf], np.nan).dropna()

# Print the combined data before further analysis
print("Combined Data Before Dropping NA:")
print(combined_data.head(30))

# Print missing values in the combined data
print("Missing values in combined data after resampling and dropping:")
print(combined_data.isnull().sum())

# Print the head of the combined data
print("Head of combined data:")
print(combined_data.head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

# Prepare the data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length, :-1]
        y = data[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 12
X, y = create_sequences(scaled_data, seq_length)

# Split the data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_test), scaled_data.shape[1]-1)), y_test.reshape(-1, 1)), axis=1))[:, -1]
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_pred), scaled_data.shape[1]-1)), y_pred), axis=1))[:, -1]

# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(monthly_obesity_data.index[-len(y_test):], y_test_inv, label='Observed Obesity Rate')
plt.plot(monthly_obesity_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

# Calculate and print the Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load datasets
def import_dataset(file_id, name, encoding='utf-8'):
    url = f"https://drive.google.com/uc?id={file_id}"
    return pd.read_csv(url, encoding=encoding, low_memory=False)

# Load the nutrition data
nutrition = import_dataset("1nxW91Jp65jQOdR_2_FJl1XaXDzhiHzgC", "nutrition")

# Filter for obesity data
obesity_data = nutrition[(nutrition['Topic'] == 'Obesity / Weight Status') &
                         (nutrition['Question'].str.contains('obesity', case=False))]

# Extract relevant columns and rename them
obesity_data = obesity_data[['YearStart', 'Data_Value']]
obesity_data.columns = ['Year', 'Obesity_Rate']

# Average the yearly data
obesity_data = obesity_data.groupby('Year').mean().reset_index()

# Apply a rolling mean to smooth the data
obesity_data['Obesity_Rate'] = obesity_data['Obesity_Rate'].rolling(window=3, center=True).mean()

# Generate monthly dates
monthly_dates = pd.date_range(start=f"{obesity_data['Year'].min()}-01-01", end=f"{obesity_data['Year'].max()}-12-31", freq='M')

# Interpolate to generate monthly obesity rates from yearly data
obesity_data.set_index('Year', inplace=True)
obesity_data = obesity_data.reindex(range(obesity_data.index.min(), obesity_data.index.max() + 1)).interpolate()

# Repeat each yearly rate for 12 months to create a monthly series
monthly_obesity_rate = np.repeat(obesity_data['Obesity_Rate'].values, 12)

# Add some noise to the monthly obesity rate
np.random.seed(42)
monthly_obesity_rate += np.random.normal(scale=0.5, size=monthly_obesity_rate.shape)

# Truncate or extend monthly_dates to match the length of monthly_obesity_rate
monthly_dates = monthly_dates[:len(monthly_obesity_rate)]

# Create a DataFrame for the monthly obesity data
monthly_obesity_data = pd.DataFrame({
    'DATE': monthly_dates,
    'Obesity_Rate': monthly_obesity_rate
})

# Set 'DATE' as index
monthly_obesity_data.set_index('DATE', inplace=True)

# Load the unemployment data
unemployment_new = import_dataset("16HmWmtgK8MM3RQ3Ed_R0-gRGqP67iDT7", "unemployment")

# Convert 'DATE' column to datetime format
unemployment_new['DATE'] = pd.to_datetime(unemployment_new['DATE'])

# Set 'DATE' as index
unemployment_new.set_index('DATE', inplace=True)

# Filter the unemployment data to match the date range of the obesity data
unemployment_filtered = unemployment_new.loc[monthly_obesity_data.index.min():monthly_obesity_data.index.max()]

# Resample unemployment data to monthly frequency and fill missing values
monthly_unemployment_rate = unemployment_filtered['LNS14000036'].resample('M').mean().interpolate()

# Load additional datasets
meat_stats_meat_production = import_dataset("1xZqVE4caTO4H60FFIP7Z5Lx1xCz0UuSd", "meat_stats_meat_production")
meat_stats_cold_storage = import_dataset("1YKBAaJKN_-RRp789RJ4wNYnpO40GyQqD", "meat_stats_cold_storage")
all_stocks = import_dataset("1hY7xiB-84DbqWhsZAazfwGbgtVxnBzDJ", "all_stocks")
stock_descriptions = import_dataset("1AzU_F3Usfx_iYRLH1KAMMaNZ3-9y0AwK", "stock_descriptions")

# Convert 'Date' columns to datetime format
meat_stats_meat_production['Date'] = pd.to_datetime(meat_stats_meat_production['Date'], format='%b-%Y')
meat_stats_cold_storage['Date'] = pd.to_datetime(meat_stats_cold_storage['Date'], format='%b-%Y')
all_stocks['Date'] = pd.to_datetime(all_stocks['Date-Time'])

# Set 'Date' as index
meat_stats_meat_production.set_index('Date', inplace=True)
meat_stats_cold_storage.set_index('Date', inplace=True)
all_stocks.set_index('Date', inplace=True)

# Convert 'Production' to numeric
meat_stats_meat_production['Production'] = meat_stats_meat_production['Production'].str.replace(',', '').astype(float)

# Handle missing values
meat_stats_meat_production = meat_stats_meat_production.dropna()
meat_stats_cold_storage = meat_stats_cold_storage.dropna()

# Filter data for 'Red Meat' and 'Poultry'
red_meat_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Red Meat', 'Production']
poultry_production = meat_stats_meat_production.loc[meat_stats_meat_production['Type of Meat'] == 'Poultry', 'Production']

# Filter fast food stocks
fast_food_stocks = stock_descriptions[stock_descriptions['Industry'].isin(['RETAIL-EATING PLACES', 'RETAIL-EATING & DRINKING PLACES'])]
fast_food_stock_prices = all_stocks[all_stocks['Ticker_Symbol'].isin(fast_food_stocks['Symbol'])]

# Resample to monthly data
monthly_fast_food_stock_prices = fast_food_stock_prices['Close'].resample('M').mean()
red_meat_production_monthly = red_meat_production.resample('M').mean()
poultry_production_monthly = poultry_production.resample('M').mean()

# Aggregate cold storage data by month and type of meat
cold_storage_agg = meat_stats_cold_storage.groupby([pd.Grouper(freq='M'), 'Type_Of_Meat'])['Weight'].mean().unstack(fill_value=0)
cold_storage_agg.columns = ['ColdStorageRedMeat', 'ColdStoragePoultry']

# Align both time series to the same time range
start_date = max(monthly_fast_food_stock_prices.index.min(), red_meat_production_monthly.index.min(), poultry_production_monthly.index.min(), cold_storage_agg.index.min())
end_date = min(monthly_fast_food_stock_prices.index.max(), red_meat_production_monthly.index.max(), poultry_production_monthly.index.max(), cold_storage_agg.index.max())

monthly_fast_food_stock_prices = monthly_fast_food_stock_prices.loc[start_date:end_date]
red_meat_production_monthly = red_meat_production_monthly.loc[start_date:end_date]
poultry_production_monthly = poultry_production_monthly.loc[start_date:end_date]
cold_storage_agg = cold_storage_agg.loc[start_date:end_date]

# Combine all data
combined_data = pd.concat([monthly_obesity_data, monthly_unemployment_rate, monthly_fast_food_stock_prices,
                           red_meat_production_monthly, poultry_production_monthly, cold_storage_agg], axis=1).dropna()

# Print the combined data before further analysis
print("Combined Data Before Dropping NA:")
print(combined_data.head(30))

# Print missing values in the combined data
print("Missing values in combined data after resampling and dropping:")
print(combined_data.isnull().sum())

# Print the head of the combined data
print("Head of combined data:")
print(combined_data.head())

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(combined_data)

# Prepare the data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length, :-1]
        y = data[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 12
X, y = create_sequences(scaled_data, seq_length)

# Split the data into train and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and test data
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_test), scaled_data.shape[1]-1)), y_test.reshape(-1, 1)), axis=1))[:, -1]
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_pred), scaled_data.shape[1]-1)), y_pred), axis=1))[:, -1]

# Plot the observed and predicted obesity rates
plt.figure(figsize=(10, 6))
plt.plot(combined_data.index[-len(y_test):], y_test_inv, label='Observed Obesity Rate')
plt.plot(combined_data.index[-len(y_test):], y_pred_inv, label='Predicted Obesity Rate', linestyle='--')
plt.title('Observed and Predicted Obesity Rate')
plt.xlabel('Date')
plt.ylabel('Obesity Rate')
plt.legend()
plt.show()

# Calculate and print the Mean Squared Error
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")


